<a href="https://colab.research.google.com/github/Heimeinsheu/software_defects_phase1/blob/main/Dissertation_ShuffleSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Shuffle Split**
Unlike KFold, ShuffleSplit leaves out a percentage of the data, not to be used in the train or validation sets. To do so we must decide what the train and test sizes are, as well as the number of splits.

#Importing Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit, cross_val_score

from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Mounted at /content/drive


In [2]:
kc2_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/softwareDefect.csv")
jm1_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/jm1.csv")
ar1_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/ar1.csv")
ar3_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/ar3.csv")
ar4_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/ar4.csv")
ar5_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/ar5.csv")
ar6_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/ar6.csv")
kc1_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/kc1.csv")
cm1_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/cm1.csv")
pc1_df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Dissertation/pc1.csv")

#Attribute Information:
 
 	
      1. loc             :   % McCabe's line count of code
      2. v(g)            :   % McCabe "cyclomatic complexity"
      3. ev(g)           :   % McCabe "essential complexity"
      4. iv(g)           :   % McCabe "design complexity"
      5. n               :   % Halstead total operators + operands
      6. v               :   % Halstead "volume"
      7. l               :   % Halstead "program length"
      8. d               :   % Halstead "difficulty"
      9. i               :   % Halstead "intelligence"
     10. e               :   % Halstead "effort"
     11. b               :   % Halstead 
     12. t               :   % Halstead's time estimator
     13. lOCode          :   % Halstead's line count
     14. lOComment       :   % Halstead's count of lines of comments
     15. lOBlank         :   % Halstead's count of blank lines
     16. lOCodeAndComment:  
     17. uniq_Op         :   % unique operators
     18. uniq_Opnd       :   % unique operands
     19. total_Op        :   % total operators
     20. total_Opnd      :   % total operands
     21. branchCount     :   % of the flow graph
     22. problems        : {no,yes}% module has/has not one or more 
                                   % reported defects


####**Calculate Numbers of Column in each dataset**

In [3]:
print("attributes in jm1: ",jm1_df.shape[1])
print("attributes in ar1: ",ar1_df.shape[1])
print("attributes in ar3: ",ar3_df.shape[1])
print("attributes in ar4: ",ar4_df.shape[1])
print("attributes in ar5: ",ar5_df.shape[1])
print("attributes in ar6: ",ar6_df.shape[1])
print("attributes in kc1: ",kc1_df.shape[1])
print("attributes in cm1: ",cm1_df.shape[1])
print("attributes in pc1: ",pc1_df.shape[1])

attributes in jm1:  22
attributes in ar1:  30
attributes in ar3:  30
attributes in ar4:  30
attributes in ar5:  30
attributes in ar6:  30
attributes in kc1:  22
attributes in cm1:  22
attributes in pc1:  22


####**Assigning Header to each dataset**

In [4]:
# adding column name to the respective columns
kc2_df.columns =['l_count', 'cyclomatic_complexity', 'essential_complexity', 'design_complexity','n','v','l','d','i','e','b','t','lOCode','lOComment','lOBlank','lOCodeAndComment','uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount','problems']
kc1_df.columns =['l_count', 'cyclomatic_complexity', 'essential_complexity', 'design_complexity','n','v','l','d','i','e','b','t','lOCode','lOComment','lOBlank','lOCodeAndComment','uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount','problems']
cm1_df.columns =['l_count', 'cyclomatic_complexity', 'essential_complexity', 'design_complexity','n','v','l','d','i','e','b','t','lOCode','lOComment','lOBlank','lOCodeAndComment','uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount','problems']
pc1_df.columns =['l_count', 'cyclomatic_complexity', 'essential_complexity', 'design_complexity','n','v','l','d','i','e','b','t','lOCode','lOComment','lOBlank','lOCodeAndComment','uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount','problems']
jm1_df.columns =['l_count', 'cyclomatic_complexity', 'essential_complexity', 'design_complexity','n','v','l','d','i','e','b','t','lOCode','lOComment','lOBlank','lOCodeAndComment','uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount','problems']
ar1_df.columns=['total_loc','blank_loc','comment_loc','code_and_comment_loc','executable_loc','unique_operands','unique_operators','total_operands','total_operators','halstead_vocabulary','halstead_length','halstead_volume','halstead_level','halstead_difficulty','halstead_effort','halstead_error','halstead_time','branch_count','decision_count','call_pairs','condition_count','multiple_condition_count','cyclomatic_complexity','cyclomatic_density','decision_density','design_complexity','design_density','normalized_cyclomatic_complexity','formal_parameters','defects']
ar3_df.columns=['total_loc','blank_loc','comment_loc','code_and_comment_loc','executable_loc','unique_operands','unique_operators','total_operands','total_operators','halstead_vocabulary','halstead_length','halstead_volume','halstead_level','halstead_difficulty','halstead_effort','halstead_error','halstead_time','branch_count','decision_count','call_pairs','condition_count','multiple_condition_count','cyclomatic_complexity','cyclomatic_density','decision_density','design_complexity','design_density','normalized_cyclomatic_complexity','formal_parameters','defects']
ar4_df.columns=['total_loc','blank_loc','comment_loc','code_and_comment_loc','executable_loc','unique_operands','unique_operators','total_operands','total_operators','halstead_vocabulary','halstead_length','halstead_volume','halstead_level','halstead_difficulty','halstead_effort','halstead_error','halstead_time','branch_count','decision_count','call_pairs','condition_count','multiple_condition_count','cyclomatic_complexity','cyclomatic_density','decision_density','design_complexity','design_density','normalized_cyclomatic_complexity','formal_parameters','defects']
ar5_df.columns=['total_loc','blank_loc','comment_loc','code_and_comment_loc','executable_loc','unique_operands','unique_operators','total_operands','total_operators','halstead_vocabulary','halstead_length','halstead_volume','halstead_level','halstead_difficulty','halstead_effort','halstead_error','halstead_time','branch_count','decision_count','call_pairs','condition_count','multiple_condition_count','cyclomatic_complexity','cyclomatic_density','decision_density','design_complexity','design_density','normalized_cyclomatic_complexity','formal_parameters','defects']
ar6_df.columns=['total_loc','blank_loc','comment_loc','code_and_comment_loc','executable_loc','unique_operands','unique_operators','total_operands','total_operators','halstead_vocabulary','halstead_length','halstead_volume','halstead_level','halstead_difficulty','halstead_effort','halstead_error','halstead_time','branch_count','decision_count','call_pairs','condition_count','multiple_condition_count','cyclomatic_complexity','cyclomatic_density','decision_density','design_complexity','design_density','normalized_cyclomatic_complexity','formal_parameters','defects']


###**Checking type of each attribute in dataset.**

In [5]:
print(kc1_df.dtypes)

l_count                    int64
cyclomatic_complexity      int64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                    int64
uniq_Opnd                  int64
total_Op                   int64
total_Opnd                 int64
branchCount                int64
problems                    bool
dtype: object


In [6]:
print(kc2_df.dtypes)

l_count                    int64
cyclomatic_complexity    float64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                    int64
uniq_Opnd                  int64
total_Op                   int64
total_Opnd                 int64
branchCount                int64
problems                  object
dtype: object


In [7]:
kc2_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1.0,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,yes
1,415,59.0,50,51,1159,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47,106,692,467,106,yes
2,230,33.0,10,16,575,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23,67,343,232,65,yes
3,175,26.0,12,13,500,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18,58,310,190,51,yes
4,163,16.0,13,11,440,2714.77,0.03,32.25,84.14,87589.65,...,139,2,20,0,19,53,260,180,31,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,4,1.0,1,1,5,11.61,0.50,2.00,5.80,23.22,...,2,0,0,0,4,1,4,1,1,yes
517,4,1.0,1,1,4,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3,1,3,1,1,yes
518,4,1.0,1,1,4,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3,1,3,1,1,yes
519,4,1.0,1,1,5,11.61,0.67,1.50,7.74,17.41,...,2,0,0,0,3,2,3,2,1,yes


In [8]:
print(jm1_df.dtypes)

l_count                    int64
cyclomatic_complexity      int64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                   object
uniq_Opnd                 object
total_Op                  object
total_Opnd                object
branchCount               object
problems                    bool
dtype: object


In [9]:
jm1_df.loc[jm1_df['uniq_Op']=='?'].count()

l_count                  5
cyclomatic_complexity    5
essential_complexity     5
design_complexity        5
n                        5
v                        5
l                        5
d                        5
i                        5
e                        5
b                        5
t                        5
lOCode                   5
lOComment                5
lOBlank                  5
lOCodeAndComment         5
uniq_Op                  5
uniq_Opnd                5
total_Op                 5
total_Opnd               5
branchCount              5
problems                 5
dtype: int64

In [10]:
jm1_df.drop(jm1_df.loc[jm1_df['uniq_Op']=="?"].index, inplace=True)

While converting attribute having object type into numeric type. I encounter many rows contain '?' value in respective attribute having object datatype.<br>
Performing drop() on that rows.

In [11]:
jm1_df[['uniq_Op', 'uniq_Opnd','total_Op','total_Opnd','branchCount']] = jm1_df[['uniq_Op', 'uniq_Opnd','total_Op','total_Opnd','branchCount']].apply(pd.to_numeric)

In [12]:
jm1_df.iloc[141:144,]

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
141,71,5,1,4,223,1338.00,0.06,16.07,83.25,21503.57,...,57,1,11,0,15,49,118,105,9,True
143,87,10,5,4,0,0.00,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,18,True
144,19,2,1,2,43,188.87,0.12,8.64,21.87,1631.15,...,9,1,4,0,10,11,24,19,3,True


In [13]:
print(pc1_df.dtypes)

l_count                    int64
cyclomatic_complexity      int64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                    int64
uniq_Opnd                  int64
total_Op                   int64
total_Opnd                 int64
branchCount                int64
problems                    bool
dtype: object


In [14]:
print(cm1_df.dtypes)

l_count                    int64
cyclomatic_complexity      int64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                    int64
uniq_Opnd                  int64
total_Op                   int64
total_Opnd                 int64
branchCount                int64
problems                    bool
dtype: object


In [15]:
print(ar1_df.dtypes)

total_loc                             int64
blank_loc                             int64
comment_loc                           int64
code_and_comment_loc                  int64
executable_loc                        int64
unique_operands                       int64
unique_operators                      int64
total_operands                        int64
total_operators                       int64
halstead_vocabulary                   int64
halstead_length                       int64
halstead_volume                       int64
halstead_level                      float64
halstead_difficulty                 float64
halstead_effort                     float64
halstead_error                      float64
halstead_time                       float64
branch_count                          int64
decision_count                        int64
call_pairs                            int64
condition_count                       int64
multiple_condition_count              int64
cyclomatic_complexity           

In [16]:
print(ar3_df.dtypes)

total_loc                             int64
blank_loc                             int64
comment_loc                           int64
code_and_comment_loc                  int64
executable_loc                        int64
unique_operands                       int64
unique_operators                      int64
total_operands                        int64
total_operators                       int64
halstead_vocabulary                   int64
halstead_length                       int64
halstead_volume                       int64
halstead_level                      float64
halstead_difficulty                 float64
halstead_effort                     float64
halstead_error                      float64
halstead_time                       float64
branch_count                          int64
decision_count                        int64
call_pairs                            int64
condition_count                       int64
multiple_condition_count              int64
cyclomatic_complexity           

In [17]:
print(ar4_df.dtypes)

total_loc                             int64
blank_loc                             int64
comment_loc                           int64
code_and_comment_loc                  int64
executable_loc                        int64
unique_operands                       int64
unique_operators                      int64
total_operands                        int64
total_operators                       int64
halstead_vocabulary                   int64
halstead_length                       int64
halstead_volume                       int64
halstead_level                      float64
halstead_difficulty                 float64
halstead_effort                     float64
halstead_error                      float64
halstead_time                       float64
branch_count                          int64
decision_count                        int64
call_pairs                            int64
condition_count                       int64
multiple_condition_count              int64
cyclomatic_complexity           

In [18]:
print(ar5_df.dtypes)

total_loc                             int64
blank_loc                             int64
comment_loc                           int64
code_and_comment_loc                  int64
executable_loc                        int64
unique_operands                       int64
unique_operators                      int64
total_operands                        int64
total_operators                       int64
halstead_vocabulary                   int64
halstead_length                       int64
halstead_volume                       int64
halstead_level                      float64
halstead_difficulty                 float64
halstead_effort                     float64
halstead_error                      float64
halstead_time                       float64
branch_count                          int64
decision_count                        int64
call_pairs                            int64
condition_count                       int64
multiple_condition_count              int64
cyclomatic_complexity           

In [19]:
print(ar6_df.dtypes)

total_loc                             int64
blank_loc                             int64
comment_loc                           int64
code_and_comment_loc                  int64
executable_loc                        int64
unique_operands                       int64
unique_operators                      int64
total_operands                        int64
total_operators                       int64
halstead_vocabulary                   int64
halstead_length                       int64
halstead_volume                       int64
halstead_level                      float64
halstead_difficulty                 float64
halstead_effort                     float64
halstead_error                      float64
halstead_time                       float64
branch_count                        float64
decision_count                        int64
call_pairs                            int64
condition_count                       int64
multiple_condition_count              int64
cyclomatic_complexity           

####**Converting attribute type from object to bool of respective dataset**
**kc2** is the only dataset that consist object type in **"problems"** attribute.

In [20]:
kc2_df['problems'] = kc2_df['problems'].map({'yes':True ,'no':False})

In [21]:
print(kc2_df.dtypes)

l_count                    int64
cyclomatic_complexity    float64
essential_complexity       int64
design_complexity          int64
n                          int64
v                        float64
l                        float64
d                        float64
i                        float64
e                        float64
b                        float64
t                        float64
lOCode                     int64
lOComment                  int64
lOBlank                    int64
lOCodeAndComment           int64
uniq_Op                    int64
uniq_Opnd                  int64
total_Op                   int64
total_Opnd                 int64
branchCount                int64
problems                    bool
dtype: object


####**Checking for null values**

In [22]:
kc1_df.isnull().sum()

l_count                  0
cyclomatic_complexity    0
essential_complexity     0
design_complexity        0
n                        0
v                        0
l                        0
d                        0
i                        0
e                        0
b                        0
t                        0
lOCode                   0
lOComment                0
lOBlank                  0
lOCodeAndComment         0
uniq_Op                  0
uniq_Opnd                0
total_Op                 0
total_Opnd               0
branchCount              0
problems                 0
dtype: int64

In [23]:
kc2_df.isnull().sum()

l_count                  0
cyclomatic_complexity    0
essential_complexity     0
design_complexity        0
n                        0
v                        0
l                        0
d                        0
i                        0
e                        0
b                        0
t                        0
lOCode                   0
lOComment                0
lOBlank                  0
lOCodeAndComment         0
uniq_Op                  0
uniq_Opnd                0
total_Op                 0
total_Opnd               0
branchCount              0
problems                 0
dtype: int64

In [24]:
cm1_df.isnull().sum()

l_count                  0
cyclomatic_complexity    0
essential_complexity     0
design_complexity        0
n                        0
v                        0
l                        0
d                        0
i                        0
e                        0
b                        0
t                        0
lOCode                   0
lOComment                0
lOBlank                  0
lOCodeAndComment         0
uniq_Op                  0
uniq_Opnd                0
total_Op                 0
total_Opnd               0
branchCount              0
problems                 0
dtype: int64

In [25]:
pc1_df.isnull().sum()

l_count                  0
cyclomatic_complexity    0
essential_complexity     0
design_complexity        0
n                        0
v                        0
l                        0
d                        0
i                        0
e                        0
b                        0
t                        0
lOCode                   0
lOComment                0
lOBlank                  0
lOCodeAndComment         0
uniq_Op                  0
uniq_Opnd                0
total_Op                 0
total_Opnd               0
branchCount              0
problems                 0
dtype: int64

In [26]:
jm1_df.isnull().sum()

l_count                  0
cyclomatic_complexity    0
essential_complexity     0
design_complexity        0
n                        0
v                        0
l                        0
d                        0
i                        0
e                        0
b                        0
t                        0
lOCode                   0
lOComment                0
lOBlank                  0
lOCodeAndComment         0
uniq_Op                  0
uniq_Opnd                0
total_Op                 0
total_Opnd               0
branchCount              0
problems                 0
dtype: int64

In [27]:
ar1_df.isnull().sum()

total_loc                           0
blank_loc                           0
comment_loc                         0
code_and_comment_loc                0
executable_loc                      0
unique_operands                     0
unique_operators                    0
total_operands                      0
total_operators                     0
halstead_vocabulary                 0
halstead_length                     0
halstead_volume                     0
halstead_level                      0
halstead_difficulty                 0
halstead_effort                     0
halstead_error                      0
halstead_time                       0
branch_count                        0
decision_count                      0
call_pairs                          0
condition_count                     0
multiple_condition_count            0
cyclomatic_complexity               0
cyclomatic_density                  0
decision_density                    0
design_complexity                   0
design_densi

In [28]:
ar3_df.isnull().sum()

total_loc                           0
blank_loc                           0
comment_loc                         0
code_and_comment_loc                0
executable_loc                      0
unique_operands                     0
unique_operators                    0
total_operands                      0
total_operators                     0
halstead_vocabulary                 0
halstead_length                     0
halstead_volume                     0
halstead_level                      0
halstead_difficulty                 0
halstead_effort                     0
halstead_error                      0
halstead_time                       0
branch_count                        0
decision_count                      0
call_pairs                          0
condition_count                     0
multiple_condition_count            0
cyclomatic_complexity               0
cyclomatic_density                  0
decision_density                    0
design_complexity                   0
design_densi

In [29]:
ar4_df.isnull().sum()

total_loc                           0
blank_loc                           0
comment_loc                         0
code_and_comment_loc                0
executable_loc                      0
unique_operands                     0
unique_operators                    0
total_operands                      0
total_operators                     0
halstead_vocabulary                 0
halstead_length                     0
halstead_volume                     0
halstead_level                      0
halstead_difficulty                 0
halstead_effort                     0
halstead_error                      0
halstead_time                       0
branch_count                        0
decision_count                      0
call_pairs                          0
condition_count                     0
multiple_condition_count            0
cyclomatic_complexity               0
cyclomatic_density                  0
decision_density                    0
design_complexity                   0
design_densi

In [30]:
ar5_df.isnull().sum()

total_loc                           0
blank_loc                           0
comment_loc                         0
code_and_comment_loc                0
executable_loc                      0
unique_operands                     0
unique_operators                    0
total_operands                      0
total_operators                     0
halstead_vocabulary                 0
halstead_length                     0
halstead_volume                     0
halstead_level                      0
halstead_difficulty                 0
halstead_effort                     0
halstead_error                      0
halstead_time                       0
branch_count                        0
decision_count                      0
call_pairs                          0
condition_count                     0
multiple_condition_count            0
cyclomatic_complexity               0
cyclomatic_density                  0
decision_density                    0
design_complexity                   0
design_densi

In [31]:
ar6_df.isnull().sum()

total_loc                           0
blank_loc                           0
comment_loc                         0
code_and_comment_loc                0
executable_loc                      0
unique_operands                     0
unique_operators                    0
total_operands                      0
total_operators                     0
halstead_vocabulary                 0
halstead_length                     0
halstead_volume                     0
halstead_level                      0
halstead_difficulty                 0
halstead_effort                     0
halstead_error                      0
halstead_time                       0
branch_count                        0
decision_count                      0
call_pairs                          0
condition_count                     0
multiple_condition_count            0
cyclomatic_complexity               0
cyclomatic_density                  0
decision_density                    0
design_complexity                   0
design_densi

# Base measures: 
mu1             = number of unique operators<br>
mu2             = number of unique operands<br>
N1              = total occurrences of operators<br>
N2              = total occurrences of operands<br>
length     = N  = N1 + N2<br>
vocabulary = mu = mu1 + mu2<br><br>
**Constants set for each function:**<br>
mu1' =  2 = potential operator count (just the function name and the "return" operator)<br>
mu2'= potential operand count. (the number of arguments to the module)<br>    
For example, the expression "return max(w+x,x+y)" has "N1=4" operators "return, max, +,+)", "N2=4" operands (w,x,x,y),"mu1=3" unique operators (return, max,+), and "mu2=3" unique operands (w,x,y).<br>
<br> 
**Derived measures:**<br>
P = volume = V = N * log2(mu) (the number of mental comparisons needed to write a program of length N)<br>
V* = volume on minimal implementation = (2 + mu2')*log2(2 + mu2')<br>
L  = program length = V*/N<br>
D  = difficulty = 1/L<br>
L' = 1/D <br>
I  = intelligence = L'*V'<br>
E  = effort to write program = V/L<br> 
T  = time to write program = E/18 seconds<br>

#**kc2 Dataset**

In [32]:
kc2_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1.0,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
1,415,59.0,50,51,1159,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47,106,692,467,106,True
2,230,33.0,10,16,575,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23,67,343,232,65,True
3,175,26.0,12,13,500,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18,58,310,190,51,True
4,163,16.0,13,11,440,2714.77,0.03,32.25,84.14,87589.65,...,139,2,20,0,19,53,260,180,31,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,4,1.0,1,1,5,11.61,0.50,2.00,5.80,23.22,...,2,0,0,0,4,1,4,1,1,True
517,4,1.0,1,1,4,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3,1,3,1,1,True
518,4,1.0,1,1,4,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3,1,3,1,1,True
519,4,1.0,1,1,5,11.61,0.67,1.50,7.74,17.41,...,2,0,0,0,3,2,3,2,1,True


##**Spiliting DataSet**

Training: 80%
Testing: 20%



In [33]:
x = kc2_df.iloc[:,:-1]
y = kc2_df.iloc[:,-1]

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [35]:
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [36]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train)  

LogisticRegression(random_state=0)

In [37]:
y_pred= classifier.predict(x_test)  
print(y_pred)

[False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False  True  True False False False False  True False False False False
 False  True False False False False False False  True False False False
 False False False False False False False  True False False  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False]


In [38]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print(cm)

[[81  3]
 [15  6]]


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8285714285714286

In [40]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8285714285714286


In [41]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [42]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85350318 0.82802548]
Average CV Score:  0.8407643312101911
Number of CV Scores used in Average:  2


In [43]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [44]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.84076433 0.81528662]
Average CV Score:  0.8195329087048832
Number of CV Scores used in Average:  3


In [45]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [46]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82802548 0.85350318 0.80254777 0.82802548]
Average CV Score:  0.8280254777070064
Number of CV Scores used in Average:  4


In [47]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [48]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87898089 0.84076433 0.84076433 0.87898089 0.81528662]
Average CV Score:  0.8509554140127389
Number of CV Scores used in Average:  5


In [49]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [50]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87898089 0.8089172  0.82802548 0.85987261 0.8089172  0.78343949]
Average CV Score:  0.8280254777070063
Number of CV Scores used in Average:  6


In [51]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [52]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.86624204 0.86624204 0.82802548 0.8089172  0.81528662
 0.76433121]
Average CV Score:  0.8216560509554139
Number of CV Scores used in Average:  7


In [53]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [54]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78343949 0.84713376 0.82802548 0.8343949  0.78343949 0.78343949
 0.77707006 0.82165605]
Average CV Score:  0.8073248407643313
Number of CV Scores used in Average:  8


In [55]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [56]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85987261 0.82165605 0.77070064 0.82165605 0.8089172  0.85987261
 0.82802548 0.87898089 0.85350318]
Average CV Score:  0.8336871903750885
Number of CV Scores used in Average:  9


In [57]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [58]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87898089 0.84076433 0.85987261 0.84713376 0.8343949  0.80254777
 0.84713376 0.82802548 0.87898089 0.80254777]
Average CV Score:  0.8420382165605096
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [59]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [60]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [61]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[83  1]
 [10 11]]


In [62]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8952380952380953


In [63]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [64]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82802548 0.8343949 ]
Average CV Score:  0.8312101910828026
Number of CV Scores used in Average:  2


In [65]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [66]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.79617834 0.79617834 0.8089172 ]
Average CV Score:  0.8004246284501062
Number of CV Scores used in Average:  3


In [67]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [68]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85987261 0.88535032 0.80254777 0.77707006]
Average CV Score:  0.8312101910828027
Number of CV Scores used in Average:  4


In [69]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [70]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.79617834 0.82802548 0.85987261 0.85350318 0.87898089]
Average CV Score:  0.8433121019108281
Number of CV Scores used in Average:  5


In [71]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [72]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.78980892 0.8089172  0.84076433 0.85350318 0.87261146]
Average CV Score:  0.8280254777070065
Number of CV Scores used in Average:  6


In [73]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [74]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77707006 0.85350318 0.74522293 0.79617834 0.8089172  0.75796178
 0.85350318]
Average CV Score:  0.7989080982711557
Number of CV Scores used in Average:  7


In [75]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [76]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85987261 0.87261146 0.86624204 0.82165605 0.85350318 0.8089172
 0.84713376 0.85987261]
Average CV Score:  0.8487261146496816
Number of CV Scores used in Average:  8


In [77]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [78]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84076433 0.85987261 0.8343949  0.84076433 0.76433121 0.82802548
 0.81528662 0.81528662 0.81528662]
Average CV Score:  0.8237791932059448
Number of CV Scores used in Average:  9


In [79]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [80]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81528662 0.86624204 0.9044586  0.84076433 0.88535032 0.82165605
 0.80254777 0.78980892 0.77070064 0.80254777]
Average CV Score:  0.8299363057324841
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [81]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [82]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [83]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[73 11]
 [ 9 12]]


In [84]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8095238095238095


In [85]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [86]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78343949 0.8089172 ]
Average CV Score:  0.7961783439490446
Number of CV Scores used in Average:  2


In [87]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [88]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82165605 0.79617834 0.77070064]
Average CV Score:  0.7961783439490446
Number of CV Scores used in Average:  3


In [89]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [90]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.78343949 0.78980892 0.78343949]
Average CV Score:  0.7898089171974523
Number of CV Scores used in Average:  4


In [91]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [92]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77070064 0.80254777 0.8089172  0.77707006 0.85987261]
Average CV Score:  0.8038216560509553
Number of CV Scores used in Average:  5


In [93]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [94]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77707006 0.75159236 0.78343949 0.78980892 0.85987261 0.77707006]
Average CV Score:  0.7898089171974522
Number of CV Scores used in Average:  6


In [95]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [96]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.80254777 0.76433121 0.84076433 0.77707006 0.78343949
 0.81528662]
Average CV Score:  0.7979981801637853
Number of CV Scores used in Average:  7


In [97]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [98]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.72611465 0.75796178 0.81528662 0.77707006 0.8089172
 0.74522293 0.80254777]
Average CV Score:  0.7794585987261147
Number of CV Scores used in Average:  8


In [99]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [100]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77070064 0.77070064 0.77707006 0.80254777 0.8089172  0.77707006
 0.78980892 0.80254777 0.75796178]
Average CV Score:  0.7841472045293703
Number of CV Scores used in Average:  9


In [101]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [102]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78980892 0.82165605 0.82802548 0.85350318 0.77707006 0.76433121
 0.8089172  0.82165605 0.8089172  0.8343949 ]
Average CV Score:  0.8108280254777072
Number of CV Scores used in Average:  10


##**Random Forest**

In [103]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [104]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [105]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[77  7]
 [ 8 13]]


In [106]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8571428571428571


In [107]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [108]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77707006 0.85987261]
Average CV Score:  0.8184713375796179
Number of CV Scores used in Average:  2


In [109]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [110]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78343949 0.85350318 0.84076433]
Average CV Score:  0.8259023354564756
Number of CV Scores used in Average:  3


In [111]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [112]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.82802548 0.84076433 0.85350318]
Average CV Score:  0.8312101910828027
Number of CV Scores used in Average:  4


In [113]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [114]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8343949  0.79617834 0.87261146 0.85987261 0.82165605]
Average CV Score:  0.8369426751592357
Number of CV Scores used in Average:  5


In [115]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [116]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.85987261 0.81528662 0.79617834 0.77707006 0.84076433]
Average CV Score:  0.8152866242038216
Number of CV Scores used in Average:  6


In [117]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [118]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8343949  0.85350318 0.84713376 0.84076433 0.79617834 0.84076433
 0.86624204]
Average CV Score:  0.8398544131028206
Number of CV Scores used in Average:  7


In [119]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [120]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84713376 0.88535032 0.80254777 0.84076433 0.84713376 0.77707006
 0.81528662 0.8343949 ]
Average CV Score:  0.8312101910828026
Number of CV Scores used in Average:  8


In [121]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [122]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8089172  0.75796178 0.84076433 0.78980892 0.86624204 0.85350318
 0.81528662 0.84076433 0.82802548]
Average CV Score:  0.8223637650389244
Number of CV Scores used in Average:  9


In [123]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [124]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85350318 0.84076433 0.85350318 0.82165605 0.81528662 0.8343949
 0.8089172  0.77707006 0.84076433 0.80254777]
Average CV Score:  0.8248407643312102
Number of CV Scores used in Average:  10


##**Bagging**

In [125]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [126]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.9855769230769231
Test data accuracy: 0.8380952380952381


In [127]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [128]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84076433 0.86624204]
Average CV Score:  0.8535031847133758
Number of CV Scores used in Average:  2


In [129]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [130]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8089172  0.84076433 0.82165605]
Average CV Score:  0.823779193205945
Number of CV Scores used in Average:  3


In [131]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [132]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81528662 0.8343949  0.84076433 0.85987261]
Average CV Score:  0.8375796178343949
Number of CV Scores used in Average:  4


In [133]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [134]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84076433 0.82802548 0.81528662 0.80254777 0.84076433]
Average CV Score:  0.8254777070063695
Number of CV Scores used in Average:  5


In [135]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [136]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80254777 0.85350318 0.85987261 0.82802548 0.84076433 0.8089172 ]
Average CV Score:  0.8322717622080681
Number of CV Scores used in Average:  6


In [137]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [138]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82165605 0.82802548 0.76433121 0.84713376 0.85350318 0.84713376
 0.82802548]
Average CV Score:  0.8271155595996361
Number of CV Scores used in Average:  7


In [139]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [140]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82802548 0.82165605 0.85350318 0.84076433 0.84713376 0.8089172
 0.84713376 0.82802548]
Average CV Score:  0.8343949044585988
Number of CV Scores used in Average:  8


In [141]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [142]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85350318 0.80254777 0.8343949  0.85350318 0.8089172  0.81528662
 0.87261146 0.81528662 0.79617834]
Average CV Score:  0.8280254777070064
Number of CV Scores used in Average:  9


In [143]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [144]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75796178 0.80254777 0.78343949 0.84076433 0.80254777 0.81528662
 0.85350318 0.82165605 0.77070064 0.8343949 ]
Average CV Score:  0.8082802547770701
Number of CV Scores used in Average:  10


#**kc1 Dataset**

In [145]:
kc1_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
1,83,11,1,11,171,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18,25,107,64,21,True
2,46,8,6,8,141,769.78,0.07,14.86,51.81,11436.73,...,37,2,5,0,16,28,89,52,15,True
3,25,3,1,3,58,254.75,0.11,9.35,27.25,2381.95,...,21,0,2,0,11,10,41,17,5,True
4,43,3,1,3,115,569.73,0.09,11.27,50.53,6423.73,...,35,2,4,0,11,20,74,41,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,19,2,1,2,40,175.69,0.15,6.82,25.77,1197.90,...,12,1,2,0,10,11,25,15,3,False
2104,23,3,3,3,60,278.63,0.10,9.69,28.75,2700.58,...,18,1,2,0,12,13,39,21,5,False
2105,2,1,1,1,4,8.00,0.67,1.50,5.33,12.00,...,0,0,0,0,3,1,3,1,1,False
2106,13,1,1,1,17,60.94,0.25,4.00,15.24,243.78,...,6,0,5,0,6,6,9,8,1,False


####**Spiliting Dataset**
training: 80% testing:20%

In [146]:
x = kc1_df.iloc[:,:-1]
y = kc1_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [147]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.8483412322274881
Confusion Matrix: 
 [[347   6]
 [ 58  11]]


In [148]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [149]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.835703  0.8436019]
Average CV Score:  0.839652448657188
Number of CV Scores used in Average:  2


In [150]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [151]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85150079 0.83254344 0.85781991]
Average CV Score:  0.8472880463401791
Number of CV Scores used in Average:  3


In [152]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [153]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85150079 0.85624013 0.835703   0.83728278]
Average CV Score:  0.8451816745655607
Number of CV Scores used in Average:  4


In [154]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [155]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8436019  0.84992101 0.85624013 0.81674566 0.83886256]
Average CV Score:  0.8410742496050553
Number of CV Scores used in Average:  5


In [156]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [157]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83412322 0.84992101 0.83728278 0.82780411 0.81832543 0.84834123]
Average CV Score:  0.835966298051606
Number of CV Scores used in Average:  6


In [158]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [159]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84992101 0.85939968 0.85150079 0.85624013 0.83412322 0.85308057
 0.84202212]
Average CV Score:  0.8494696456781765
Number of CV Scores used in Average:  7


In [160]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [161]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86255924 0.85939968 0.85781991 0.84202212 0.85781991 0.82148499
 0.85466035 0.84518167]
Average CV Score:  0.8501184834123223
Number of CV Scores used in Average:  8


In [162]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [163]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83886256 0.86097946 0.83728278 0.85781991 0.84518167 0.85150079
 0.83728278 0.83254344 0.85939968]
Average CV Score:  0.8467614533965244
Number of CV Scores used in Average:  9


In [164]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [165]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83412322 0.87045814 0.85150079 0.81674566 0.84518167 0.85466035
 0.86097946 0.83886256 0.84518167 0.85781991]
Average CV Score:  0.8475513428120063
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [166]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [167]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [168]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[348   5]
 [ 60   9]]


In [169]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8459715639810427


In [170]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [171]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83412322 0.84992101]
Average CV Score:  0.8420221169036335
Number of CV Scores used in Average:  2


In [172]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [173]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84834123 0.86255924 0.86255924]
Average CV Score:  0.8578199052132701
Number of CV Scores used in Average:  3


In [174]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [175]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85308057 0.84834123 0.8436019  0.86413902]
Average CV Score:  0.8522906793048973
Number of CV Scores used in Average:  4


In [176]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [177]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.835703   0.85781991 0.86097946 0.86729858 0.85781991]
Average CV Score:  0.8559241706161137
Number of CV Scores used in Average:  5


In [178]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [179]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83412322 0.84518167 0.83886256 0.85308057 0.82780411 0.83886256]
Average CV Score:  0.8396524486571879
Number of CV Scores used in Average:  6


In [180]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [181]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85781991 0.84202212 0.86729858 0.84518167 0.85781991 0.85939968
 0.84202212]
Average CV Score:  0.8530805687203792
Number of CV Scores used in Average:  7


In [182]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [183]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86413902 0.85308057 0.84992101 0.86413902 0.86413902 0.86097946
 0.84202212 0.85939968]
Average CV Score:  0.8572274881516588
Number of CV Scores used in Average:  8


In [184]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [185]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8657188  0.84518167 0.86729858 0.83886256 0.85308057 0.84992101
 0.85624013 0.84518167 0.83254344]
Average CV Score:  0.8504476040021064
Number of CV Scores used in Average:  9


In [186]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [187]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85466035 0.86729858 0.86413902 0.86097946 0.84676145 0.87045814
 0.85150079 0.86413902 0.87677725 0.83728278]
Average CV Score:  0.8593996840442338
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [188]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [189]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [190]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[328  25]
 [ 43  26]]


In [191]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8388625592417062


In [192]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [193]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82938389 0.81674566]
Average CV Score:  0.8230647709320695
Number of CV Scores used in Average:  2


In [194]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [195]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82622433 0.82622433 0.7835703 ]
Average CV Score:  0.8120063191153238
Number of CV Scores used in Average:  3


In [196]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [197]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81674566 0.82306477 0.8056872  0.84676145]
Average CV Score:  0.8230647709320695
Number of CV Scores used in Average:  4


In [198]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [199]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80884676 0.80410742 0.8436019  0.85308057 0.83886256]
Average CV Score:  0.829699842022117
Number of CV Scores used in Average:  5


In [200]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [201]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.77409163 0.81832543 0.82780411 0.81990521 0.78830964 0.83254344]
Average CV Score:  0.8101632438125329
Number of CV Scores used in Average:  6


In [202]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [203]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80726698 0.8436019  0.84992101 0.8436019  0.84044234 0.81990521
 0.83254344]
Average CV Score:  0.8338975400586774
Number of CV Scores used in Average:  7


In [204]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [205]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82306477 0.84834123 0.82148499 0.835703   0.82780411 0.8135861
 0.83412322 0.82622433]
Average CV Score:  0.8287914691943128
Number of CV Scores used in Average:  8


In [206]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [207]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82622433 0.85466035 0.82622433 0.83254344 0.82148499 0.82464455
 0.8135861  0.82464455 0.81516588]
Average CV Score:  0.8265753905564331
Number of CV Scores used in Average:  9


In [208]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [209]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8056872  0.82622433 0.81990521 0.81832543 0.835703   0.79936809
 0.83412322 0.78515008 0.82622433 0.81516588]
Average CV Score:  0.8165876777251185
Number of CV Scores used in Average:  10


##**Random Forest**

In [210]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [211]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [212]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[345   8]
 [ 49  20]]


In [213]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8649289099526066


In [214]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [215]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83728278 0.86255924]
Average CV Score:  0.8499210110584519
Number of CV Scores used in Average:  2


In [216]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [217]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82938389 0.85624013 0.87045814]
Average CV Score:  0.8520273828330701
Number of CV Scores used in Average:  3


In [218]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [219]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84834123 0.86413902 0.83254344 0.85781991]
Average CV Score:  0.8507109004739337
Number of CV Scores used in Average:  4


In [220]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [221]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82938389 0.84992101 0.87045814 0.82938389 0.84518167]
Average CV Score:  0.844865718799368
Number of CV Scores used in Average:  5


In [222]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [223]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8436019  0.86255924 0.85781991 0.8436019  0.82306477 0.84202212]
Average CV Score:  0.845444971037388
Number of CV Scores used in Average:  6


In [224]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [225]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85308057 0.84518167 0.85781991 0.86255924 0.8436019  0.8657188
 0.85466035]
Average CV Score:  0.8546603475513427
Number of CV Scores used in Average:  7


In [226]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [227]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86887836 0.85308057 0.86729858 0.85624013 0.84676145 0.84676145
 0.87203791 0.84834123]
Average CV Score:  0.8574249605055292
Number of CV Scores used in Average:  8


In [228]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [229]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87677725 0.83728278 0.85939968 0.83886256 0.83886256 0.835703
 0.87045814 0.85781991 0.84044234]
Average CV Score:  0.8506231349833245
Number of CV Scores used in Average:  9


In [230]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [231]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84676145 0.84992101 0.84518167 0.85150079 0.83886256 0.84992101
 0.83728278 0.84992101 0.87677725 0.86097946]
Average CV Score:  0.8507109004739337
Number of CV Scores used in Average:  10


##**Bagging**

In [232]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [233]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.9899169632265717
Test data accuracy: 0.8175355450236966


In [234]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [235]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83728278 0.86413902]
Average CV Score:  0.8507109004739336
Number of CV Scores used in Average:  2


In [236]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [237]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82306477 0.85150079 0.84992101]
Average CV Score:  0.841495523959979
Number of CV Scores used in Average:  3


In [238]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [239]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86729858 0.85624013 0.86887836 0.88309637]
Average CV Score:  0.8688783570300158
Number of CV Scores used in Average:  4


In [240]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [241]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85150079 0.84202212 0.84992101 0.84834123 0.84044234]
Average CV Score:  0.8464454976303317
Number of CV Scores used in Average:  5


In [242]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [243]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85150079 0.86097946 0.84992101 0.83096367 0.87993681 0.87203791]
Average CV Score:  0.8575566087414429
Number of CV Scores used in Average:  6


In [244]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [245]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83728278 0.82464455 0.85150079 0.86729858 0.84518167 0.84834123
 0.85624013]
Average CV Score:  0.8472128187767998
Number of CV Scores used in Average:  7


In [246]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [247]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85308057 0.87835703 0.82938389 0.87677725 0.87677725 0.8436019
 0.85308057 0.85624013]
Average CV Score:  0.8584123222748814
Number of CV Scores used in Average:  8


In [248]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [249]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84044234 0.85308057 0.87677725 0.86255924 0.86255924 0.8436019
 0.85624013 0.84044234 0.85939968]
Average CV Score:  0.8550114095137792
Number of CV Scores used in Average:  9


In [250]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [251]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86413902 0.83254344 0.83254344 0.84518167 0.83096367 0.8657188
 0.84834123 0.86255924 0.8657188  0.84518167]
Average CV Score:  0.8492890995260662
Number of CV Scores used in Average:  10


#**cm1 Dataset**

In [252]:
cm1_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
1,24,5,1,3,63,309.13,0.11,9.50,32.54,2936.77,...,1,0,6,0,15,15,44,19,9,False
2,20,4,4,2,47,215.49,0.06,16.00,13.47,3447.89,...,0,0,3,0,16,8,31,16,7,False
3,24,6,6,2,72,346.13,0.06,17.33,19.97,5999.58,...,0,0,3,0,16,12,46,26,11,False
4,24,6,6,2,72,346.13,0.06,17.33,19.97,5999.58,...,0,0,3,0,16,12,46,26,11,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,47,3,1,3,256,1563.78,0.04,28.00,55.85,43785.90,...,2,13,2,0,23,46,144,112,5,True
493,24,4,3,3,107,587.63,0.05,19.13,30.72,11241.58,...,1,7,4,0,22,23,67,40,7,True
494,82,11,3,10,475,3155.83,0.02,44.71,70.59,141084.24,...,9,59,35,0,32,68,285,190,21,True
495,10,2,1,1,32,150.41,0.15,6.50,23.14,977.69,...,1,12,4,0,13,13,19,13,3,True


####**Spiliting Dataset**
training: 80% testing:20%

In [253]:
x = cm1_df.iloc[:,:-1]
y = cm1_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [254]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.93
Confusion Matrix: 
 [[92  2]
 [ 5  1]]


In [255]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [256]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87333333 0.87333333]
Average CV Score:  0.8733333333333333
Number of CV Scores used in Average:  2


In [257]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [258]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.85333333 0.92666667 0.88      ]
Average CV Score:  0.8866666666666667
Number of CV Scores used in Average:  3


In [259]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [260]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87333333 0.9        0.91333333 0.92      ]
Average CV Score:  0.9016666666666666
Number of CV Scores used in Average:  4


In [261]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [262]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88666667 0.88       0.9        0.91333333 0.88      ]
Average CV Score:  0.892
Number of CV Scores used in Average:  5


In [263]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [264]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88       0.9        0.91333333 0.91333333 0.92666667 0.9       ]
Average CV Score:  0.9055555555555556
Number of CV Scores used in Average:  6


In [265]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [266]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91333333 0.89333333 0.88666667 0.89333333 0.87333333 0.92666667
 0.92      ]
Average CV Score:  0.900952380952381
Number of CV Scores used in Average:  7


In [267]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [268]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86       0.88       0.89333333 0.94       0.89333333 0.86
 0.90666667 0.86666667]
Average CV Score:  0.8875
Number of CV Scores used in Average:  8


In [269]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [270]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87333333 0.91333333 0.88       0.92       0.91333333 0.91333333
 0.9        0.84       0.87333333]
Average CV Score:  0.8918518518518519
Number of CV Scores used in Average:  9


In [271]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [272]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89333333 0.92       0.85333333 0.88666667 0.87333333 0.87333333
 0.87333333 0.88       0.91333333 0.90666667]
Average CV Score:  0.8873333333333333
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [273]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [274]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [275]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[93  1]
 [ 6  0]]


In [276]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.93


In [277]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [278]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.87333333]
Average CV Score:  0.8866666666666667
Number of CV Scores used in Average:  2


In [279]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [280]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92666667 0.89333333 0.9       ]
Average CV Score:  0.9066666666666666
Number of CV Scores used in Average:  3


In [281]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [282]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.88666667 0.92666667 0.92      ]
Average CV Score:  0.9083333333333333
Number of CV Scores used in Average:  4


In [283]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [284]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89333333 0.9        0.91333333 0.87333333 0.92666667]
Average CV Score:  0.9013333333333333
Number of CV Scores used in Average:  5


In [285]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [286]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.9        0.92       0.92       0.85333333 0.90666667]
Average CV Score:  0.8999999999999999
Number of CV Scores used in Average:  6


In [287]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [288]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90666667 0.86       0.90666667 0.88       0.92666667 0.91333333
 0.86      ]
Average CV Score:  0.8933333333333333
Number of CV Scores used in Average:  7


In [289]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [290]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.87333333 0.93333333 0.86666667 0.91333333 0.9
 0.90666667 0.91333333]
Average CV Score:  0.9008333333333334
Number of CV Scores used in Average:  8


In [291]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [292]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91333333 0.91333333 0.91333333 0.88666667 0.92       0.92
 0.92       0.88       0.92      ]
Average CV Score:  0.9096296296296297
Number of CV Scores used in Average:  9


In [293]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [294]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92666667 0.88666667 0.91333333 0.9        0.89333333 0.87333333
 0.9        0.92666667 0.88       0.91333333]
Average CV Score:  0.9013333333333335
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [295]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [296]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [297]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[87  7]
 [ 5  1]]


In [298]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.88


In [299]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [300]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84       0.83333333]
Average CV Score:  0.8366666666666667
Number of CV Scores used in Average:  2


In [301]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [302]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84666667 0.82666667 0.84      ]
Average CV Score:  0.8377777777777777
Number of CV Scores used in Average:  3


In [303]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [304]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86       0.83333333 0.82       0.8       ]
Average CV Score:  0.8283333333333334
Number of CV Scores used in Average:  4


In [305]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [306]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82       0.83333333 0.84666667 0.86666667 0.82      ]
Average CV Score:  0.8373333333333333
Number of CV Scores used in Average:  5


In [307]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [308]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.79333333 0.87333333 0.84666667 0.84666667 0.82       0.84666667]
Average CV Score:  0.8377777777777777
Number of CV Scores used in Average:  6


In [309]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [310]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.86       0.82       0.84       0.83333333 0.83333333
 0.82666667]
Average CV Score:  0.8352380952380951
Number of CV Scores used in Average:  7


In [311]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [312]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82666667 0.80666667 0.86       0.85333333 0.8        0.80666667
 0.84666667 0.88666667]
Average CV Score:  0.8358333333333333
Number of CV Scores used in Average:  8


In [313]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [314]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82666667 0.87333333 0.85333333 0.82       0.88666667 0.84
 0.83333333 0.87333333 0.87333333]
Average CV Score:  0.8533333333333333
Number of CV Scores used in Average:  9


In [315]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [316]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88       0.8        0.82666667 0.86666667 0.78       0.78
 0.86666667 0.8        0.88       0.84666667]
Average CV Score:  0.8326666666666667
Number of CV Scores used in Average:  10


##**Random Forest**

In [317]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [318]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [319]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[94  0]
 [ 4  2]]


In [320]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.96


In [321]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [322]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.88666667]
Average CV Score:  0.8933333333333333
Number of CV Scores used in Average:  2


In [323]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [324]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88666667 0.93333333 0.85333333]
Average CV Score:  0.8911111111111111
Number of CV Scores used in Average:  3


In [325]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [326]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89333333 0.91333333 0.90666667 0.9       ]
Average CV Score:  0.9033333333333333
Number of CV Scores used in Average:  4


In [327]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [328]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.82666667 0.88666667 0.89333333 0.88666667]
Average CV Score:  0.8786666666666667
Number of CV Scores used in Average:  5


In [329]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [330]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92666667 0.88666667 0.88666667 0.91333333 0.89333333 0.88      ]
Average CV Score:  0.8977777777777778
Number of CV Scores used in Average:  6


In [331]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [332]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92666667 0.86666667 0.86       0.86       0.86666667 0.84666667
 0.85333333]
Average CV Score:  0.8685714285714285
Number of CV Scores used in Average:  7


In [333]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [334]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.92       0.92       0.89333333 0.88666667 0.88
 0.88666667 0.90666667]
Average CV Score:  0.8908333333333334
Number of CV Scores used in Average:  8


In [335]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [336]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.88       0.89333333 0.9        0.88       0.92
 0.89333333 0.88       0.88      ]
Average CV Score:  0.8918518518518519
Number of CV Scores used in Average:  9


In [337]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [338]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.91333333 0.92666667 0.91333333 0.89333333 0.9
 0.88       0.88666667 0.88666667 0.90666667]
Average CV Score:  0.9006666666666666
Number of CV Scores used in Average:  10


##**Bagging**

In [339]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [340]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.9974811083123426
Test data accuracy: 0.89


In [341]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [342]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91333333 0.9       ]
Average CV Score:  0.9066666666666667
Number of CV Scores used in Average:  2


In [343]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [344]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90666667 0.86666667 0.88666667]
Average CV Score:  0.8866666666666667
Number of CV Scores used in Average:  3


In [345]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [346]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.87333333 0.90666667 0.88666667 0.86666667]
Average CV Score:  0.8833333333333333
Number of CV Scores used in Average:  4


In [347]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [348]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.87333333 0.9        0.87333333 0.90666667]
Average CV Score:  0.8906666666666666
Number of CV Scores used in Average:  5


In [349]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [350]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89333333 0.94       0.88       0.92       0.93333333 0.89333333]
Average CV Score:  0.91
Number of CV Scores used in Average:  6


In [351]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [352]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88       0.92666667 0.89333333 0.91333333 0.89333333 0.9
 0.90666667]
Average CV Score:  0.9019047619047619
Number of CV Scores used in Average:  7


In [353]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [354]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86       0.89333333 0.9        0.81333333 0.90666667 0.87333333
 0.94       0.86666667]
Average CV Score:  0.8816666666666666
Number of CV Scores used in Average:  8


In [355]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [356]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90666667 0.9        0.88666667 0.88       0.90666667 0.92666667
 0.9        0.88       0.93333333]
Average CV Score:  0.9022222222222221
Number of CV Scores used in Average:  9


In [357]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [358]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88666667 0.89333333 0.94666667 0.90666667 0.89333333 0.87333333
 0.90666667 0.89333333 0.9        0.90666667]
Average CV Score:  0.9006666666666666
Number of CV Scores used in Average:  10


#**pc1**

In [359]:
pc1_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
1,91,9,3,2,318,2089.21,0.04,27.68,75.47,57833.24,...,80,44,11,31,29,66,192,126,17,True
2,109,21,5,18,381,2547.56,0.04,28.37,89.79,72282.68,...,97,41,12,24,28,75,229,152,38,True
3,505,106,41,82,2339,20696.93,0.01,75.93,272.58,1571506.88,...,457,71,48,49,64,397,1397,942,178,True
4,107,25,7,14,619,4282.78,0.02,52.91,80.95,226588.75,...,103,32,4,39,35,86,359,260,40,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,6,4,4,1,26,96.21,0.08,13.33,7.22,1282.82,...,6,0,0,2,10,3,18,8,7,False
1104,10,5,5,1,43,182.66,0.05,21.00,8.70,3835.88,...,10,0,0,1,14,5,28,15,9,False
1105,5,3,3,1,17,62.91,0.21,4.80,13.11,301.96,...,5,0,0,0,8,5,11,6,5,False
1106,18,8,5,5,111,613.12,0.04,22.92,26.75,14050.56,...,18,0,0,1,22,24,61,50,15,False


####**Spiliting Dataset**
training: 80% testing:20%

In [360]:
x = pc1_df.iloc[:,:-1]
y = pc1_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [361]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.9234234234234234
Confusion Matrix: 
 [[203   4]
 [ 13   2]]


In [362]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [363]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90690691 0.93093093]
Average CV Score:  0.9189189189189189
Number of CV Scores used in Average:  2


In [364]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [365]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94594595 0.92192192 0.90690691]
Average CV Score:  0.924924924924925
Number of CV Scores used in Average:  3


In [366]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [367]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92492492 0.91591592 0.92792793 0.92192192]
Average CV Score:  0.9226726726726726
Number of CV Scores used in Average:  4


In [368]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [369]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94594595 0.93393393 0.94294294 0.93393393 0.91891892]
Average CV Score:  0.9351351351351351
Number of CV Scores used in Average:  5


In [370]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [371]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91291291 0.94894895 0.95795796 0.94294294 0.94894895 0.93393393]
Average CV Score:  0.940940940940941
Number of CV Scores used in Average:  6


In [372]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [373]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9039039  0.92792793 0.93093093 0.93393393 0.94294294 0.95795796
 0.93393393]
Average CV Score:  0.9330759330759332
Number of CV Scores used in Average:  7


In [374]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [375]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92192192 0.92192192 0.94294294 0.93693694 0.93093093 0.92792793
 0.92192192 0.93993994]
Average CV Score:  0.9305555555555556
Number of CV Scores used in Average:  8


In [376]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [377]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92192192 0.93093093 0.93693694 0.93693694 0.93693694 0.91591592
 0.92492492 0.90990991 0.93093093]
Average CV Score:  0.9272605939272606
Number of CV Scores used in Average:  9


In [378]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [379]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91591592 0.94594595 0.93693694 0.93393393 0.91291291 0.91591592
 0.93093093 0.90990991 0.92192192 0.92492492]
Average CV Score:  0.924924924924925
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [380]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [381]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [382]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[207   0]
 [ 15   0]]


In [383]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9324324324324325


In [384]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [385]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93993994 0.92492492]
Average CV Score:  0.9324324324324325
Number of CV Scores used in Average:  2


In [386]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [387]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92492492 0.93093093 0.91591592]
Average CV Score:  0.923923923923924
Number of CV Scores used in Average:  3


In [388]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [389]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91891892 0.91591592 0.96096096 0.93693694]
Average CV Score:  0.9331831831831832
Number of CV Scores used in Average:  4


In [390]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [391]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92492492 0.93093093 0.93393393 0.94594595 0.91591592]
Average CV Score:  0.9303303303303304
Number of CV Scores used in Average:  5


In [392]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [393]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94294294 0.91891892 0.91891892 0.94594595 0.94294294 0.91291291]
Average CV Score:  0.9304304304304304
Number of CV Scores used in Average:  6


In [394]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [395]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93693694 0.92492492 0.92792793 0.93093093 0.93693694 0.93393393
 0.93093093]
Average CV Score:  0.9317889317889317
Number of CV Scores used in Average:  7


In [396]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [397]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93993994 0.92192192 0.93993994 0.90990991 0.93693694 0.91891892
 0.93393393 0.91591592]
Average CV Score:  0.9271771771771773
Number of CV Scores used in Average:  8


In [398]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [399]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92192192 0.92192192 0.92792793 0.92192192 0.92192192 0.91891892
 0.8978979  0.9039039  0.92192192]
Average CV Score:  0.9175842509175841
Number of CV Scores used in Average:  9


In [400]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [401]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.92792793 0.92492492 0.94594595 0.92492492 0.92492492
 0.91591592 0.91291291 0.92792793 0.91891892]
Average CV Score:  0.9255255255255257
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [402]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [403]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [404]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[202   5]
 [  9   6]]


In [405]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9369369369369369


In [406]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [407]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90690691 0.93693694]
Average CV Score:  0.9219219219219219
Number of CV Scores used in Average:  2


In [408]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [409]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91891892 0.91291291 0.91591592]
Average CV Score:  0.9159159159159159
Number of CV Scores used in Average:  3


In [410]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [411]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91291291 0.91591592 0.8978979  0.88588589]
Average CV Score:  0.9031531531531531
Number of CV Scores used in Average:  4


In [412]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [413]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92492492 0.90690691 0.92492492 0.9039039  0.90690691]
Average CV Score:  0.9135135135135135
Number of CV Scores used in Average:  5


In [414]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [415]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9009009  0.89489489 0.8978979  0.92192192 0.91591592 0.93393393]
Average CV Score:  0.9109109109109109
Number of CV Scores used in Average:  6


In [416]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [417]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92792793 0.91891892 0.91591592 0.86186186 0.92192192 0.91291291
 0.88888889]
Average CV Score:  0.906906906906907
Number of CV Scores used in Average:  7


In [418]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [419]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92792793 0.91291291 0.9039039  0.92792793 0.9009009  0.90690691
 0.92192192 0.88888889]
Average CV Score:  0.9114114114114114
Number of CV Scores used in Average:  8


In [420]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [421]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8978979  0.87987988 0.9009009  0.9009009  0.92492492 0.91291291
 0.91591592 0.91591592 0.8978979 ]
Average CV Score:  0.9052385719052387
Number of CV Scores used in Average:  9


In [422]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [423]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90990991 0.93693694 0.92192192 0.92792793 0.91591592 0.91291291
 0.92792793 0.89489489 0.91591592 0.9009009 ]
Average CV Score:  0.9165165165165166
Number of CV Scores used in Average:  10


##**Random Forest**

In [424]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [425]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [426]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[205   2]
 [ 15   0]]


In [427]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9234234234234234


In [428]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [429]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93693694 0.93693694]
Average CV Score:  0.9369369369369369
Number of CV Scores used in Average:  2


In [430]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [431]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.92492492 0.91891892]
Average CV Score:  0.924924924924925
Number of CV Scores used in Average:  3


In [432]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [433]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93993994 0.94894895 0.91291291 0.95195195]
Average CV Score:  0.9384384384384383
Number of CV Scores used in Average:  4


In [434]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [435]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94894895 0.92792793 0.93693694 0.94594595 0.94894895]
Average CV Score:  0.9417417417417419
Number of CV Scores used in Average:  5


In [436]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [437]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92792793 0.93093093 0.92792793 0.93693694 0.92492492 0.94294294]
Average CV Score:  0.9319319319319318
Number of CV Scores used in Average:  6


In [438]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [439]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93993994 0.91891892 0.94894895 0.92192192 0.93693694 0.93693694
 0.92792793]
Average CV Score:  0.9330759330759332
Number of CV Scores used in Average:  7


In [440]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [441]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93693694 0.91891892 0.92492492 0.93093093 0.92492492 0.93693694
 0.91291291 0.93093093]
Average CV Score:  0.9271771771771772
Number of CV Scores used in Average:  8


In [442]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [443]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93693694 0.94294294 0.94894895 0.92492492 0.93993994 0.94294294
 0.91891892 0.93093093 0.93993994]
Average CV Score:  0.9362696029362696
Number of CV Scores used in Average:  9


In [444]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [445]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93393393 0.91591592 0.93393393 0.93393393 0.93093093 0.93993994
 0.94894895 0.91591592 0.95195195 0.93093093]
Average CV Score:  0.9336336336336336
Number of CV Scores used in Average:  10


##**Bagging**

In [446]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [447]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.9966139954853274
Test data accuracy: 0.9234234234234234


In [448]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [449]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93693694 0.91591592]
Average CV Score:  0.9264264264264264
Number of CV Scores used in Average:  2


In [450]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [451]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.93393393 0.93693694]
Average CV Score:  0.933933933933934
Number of CV Scores used in Average:  3


In [452]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [453]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.94894895 0.92192192 0.92192192]
Average CV Score:  0.930930930930931
Number of CV Scores used in Average:  4


In [454]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [455]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.92792793 0.91591592 0.95795796 0.93693694 0.93393393]
Average CV Score:  0.9345345345345345
Number of CV Scores used in Average:  5


In [456]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [457]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.92492492 0.93693694 0.95195195 0.92492492 0.93693694]
Average CV Score:  0.9344344344344345
Number of CV Scores used in Average:  6


In [458]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [459]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.92792793 0.93693694 0.93393393 0.93093093 0.93993994
 0.94294294]
Average CV Score:  0.9347919347919348
Number of CV Scores used in Average:  7


In [460]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [461]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93993994 0.93993994 0.94594595 0.93393393 0.92192192 0.93693694
 0.93393393 0.93993994]
Average CV Score:  0.9365615615615616
Number of CV Scores used in Average:  8


In [462]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [463]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.96096096 0.92192192 0.94594595 0.91891892 0.93993994 0.93993994
 0.93093093 0.92792793 0.92792793]
Average CV Score:  0.934934934934935
Number of CV Scores used in Average:  9


In [464]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [465]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93093093 0.93393393 0.93693694 0.90690691 0.93693694 0.91891892
 0.94594595 0.93993994 0.94894895 0.93393393]
Average CV Score:  0.9333333333333333
Number of CV Scores used in Average:  10


#**jm1**

In [466]:
jm1_df

,l_count,cyclomatic_complexity,essential_complexity,design_complexity,n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1,1,1,1,1,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
1,72,7,1,6,198,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
2,190,3,1,3,600,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
3,37,4,1,4,126,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True
4,31,2,1,2,111,582.52,0.08,12.25,47.55,7135.87,...,19,0,5,0,14,24,69,42,3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10879,18,4,1,4,52,241.48,0.14,7.33,32.93,1770.86,...,13,0,2,0,10,15,30,22,7,False
10880,9,2,1,2,30,129.66,0.12,8.25,15.72,1069.68,...,5,0,2,0,12,8,19,11,3,False
10881,42,4,1,2,103,519.57,0.04,26.40,19.68,13716.72,...,29,1,10,0,18,15,59,44,7,False
10882,10,1,1,1,36,147.15,0.12,8.44,17.44,1241.57,...,6,0,2,0,9,8,21,15,1,False


####**Spiliting Dataset**
training: 80% testing:20%

In [467]:
x = jm1_df.iloc[:,:-1]
y = jm1_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [468]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.8143382352941176
Confusion Matrix: 
 [[1731   45]
 [ 359   41]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [469]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [470]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73958333 0.75153186]
Average CV Score:  0.7455575980392157
Number of CV Scores used in Average:  2


In [471]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [472]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78492647 0.74509804 0.76501225]
Average CV Score:  0.7650122549019608
Number of CV Scores used in Average:  3


In [473]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [474]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80453431 0.76072304 0.75827206 0.81127451]
Average CV Score:  0.7837009803921569
Number of CV Scores used in Average:  4


In [475]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [476]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80147059 0.76746324 0.74356618 0.74356618 0.80851716]
Average CV Score:  0.7729166666666667
Number of CV Scores used in Average:  5


In [477]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [478]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8192402  0.74509804 0.74908088 0.80300245 0.81035539 0.77052696]
Average CV Score:  0.7828839869281046
Number of CV Scores used in Average:  6


In [479]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [480]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76194853 0.79656863 0.74417892 0.80729167 0.80729167 0.7689951
 0.80514706]
Average CV Score:  0.7844887955182072
Number of CV Scores used in Average:  7


In [481]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [482]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81403186 0.81495098 0.75765931 0.78094363 0.79442402 0.80606618
 0.74969363 0.8036152 ]
Average CV Score:  0.7901731004901961
Number of CV Scores used in Average:  8


In [483]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [484]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75398284 0.79595588 0.80300245 0.80453431 0.74785539 0.79871324
 0.75949755 0.74142157 0.81158088]
Average CV Score:  0.7796160130718954
Number of CV Scores used in Average:  9


In [485]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [486]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76286765 0.74754902 0.79166667 0.74295343 0.80055147 0.76102941
 0.74693627 0.81066176 0.78921569 0.79779412]
Average CV Score:  0.7751225490196079
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [487]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [488]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [489]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[1741   35]
 [ 366   34]]


In [490]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8157169117647058


In [491]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [492]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80821078 0.79534314]
Average CV Score:  0.8017769607843137
Number of CV Scores used in Average:  2


In [493]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [494]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81188725 0.80147059 0.80729167]
Average CV Score:  0.8068831699346405
Number of CV Scores used in Average:  3


In [495]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [496]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80790441 0.8192402  0.80453431 0.81709559]
Average CV Score:  0.8121936274509804
Number of CV Scores used in Average:  4


In [497]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [498]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81648284 0.80208333 0.80729167 0.80882353 0.80974265]
Average CV Score:  0.8088848039215686
Number of CV Scores used in Average:  5


In [499]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [500]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80545343 0.81280637 0.80790441 0.81188725 0.81127451 0.80300245]
Average CV Score:  0.808721405228758
Number of CV Scores used in Average:  6


In [501]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [502]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80330882 0.80821078 0.81678922 0.8057598  0.80851716 0.81617647
 0.81035539]
Average CV Score:  0.8098739495798319
Number of CV Scores used in Average:  7


In [503]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [504]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80637255 0.79871324 0.80759804 0.81801471 0.80943627 0.80637255
 0.80729167 0.80943627]
Average CV Score:  0.8079044117647058
Number of CV Scores used in Average:  8


In [505]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [506]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81004902 0.81893382 0.80790441 0.8091299  0.81035539 0.79810049
 0.81004902 0.8091299  0.80208333]
Average CV Score:  0.8084150326797386
Number of CV Scores used in Average:  9


In [507]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [508]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81127451 0.81096814 0.81096814 0.8125     0.80116422 0.80330882
 0.81433824 0.80392157 0.80698529 0.80759804]
Average CV Score:  0.8083026960784313
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [509]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [510]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [511]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[1507  269]
 [ 253  147]]


In [512]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.7601102941176471


In [513]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [514]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.74448529 0.74908088]
Average CV Score:  0.7467830882352942
Number of CV Scores used in Average:  2


In [515]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [516]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75183824 0.76286765 0.7689951 ]
Average CV Score:  0.7612336601307189
Number of CV Scores used in Average:  3


In [517]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [518]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.74264706 0.7567402  0.75765931 0.75122549]
Average CV Score:  0.7520680147058825
Number of CV Scores used in Average:  4


In [519]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [520]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.74448529 0.75183824 0.75428922 0.75765931 0.7567402 ]
Average CV Score:  0.7530024509803922
Number of CV Scores used in Average:  5


In [521]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [522]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76715686 0.7567402  0.76470588 0.74724265 0.77267157 0.73590686]
Average CV Score:  0.757404003267974
Number of CV Scores used in Average:  6


In [523]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [524]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75735294 0.74632353 0.75459559 0.75490196 0.73835784 0.75245098
 0.75398284]
Average CV Score:  0.7511379551820728
Number of CV Scores used in Average:  7


In [525]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [526]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75582108 0.74448529 0.76685049 0.74540441 0.75490196 0.75091912
 0.76256127 0.75306373]
Average CV Score:  0.7542509191176471
Number of CV Scores used in Average:  8


In [527]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [528]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75704657 0.74908088 0.7533701  0.7533701  0.75612745 0.75122549
 0.75796569 0.75980392 0.75704657]
Average CV Score:  0.7550040849673202
Number of CV Scores used in Average:  9


In [529]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [530]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.74448529 0.74969363 0.75551471 0.76409314 0.7567402  0.74479167
 0.74785539 0.7533701  0.74203431 0.74724265]
Average CV Score:  0.7505821078431373
Number of CV Scores used in Average:  10


##**Random Forest**

In [531]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [532]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [533]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[1693   83]
 [ 318   82]]


In [534]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8157169117647058


In [535]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [536]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.82230392 0.80729167]
Average CV Score:  0.814797794117647
Number of CV Scores used in Average:  2


In [537]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [538]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81433824 0.80116422 0.81372549]
Average CV Score:  0.8097426470588235
Number of CV Scores used in Average:  3


In [539]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [540]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8057598  0.80147059 0.80606618 0.8036152 ]
Average CV Score:  0.8042279411764707
Number of CV Scores used in Average:  4


In [541]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [542]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80208333 0.80453431 0.8091299  0.80422794 0.80790441]
Average CV Score:  0.8055759803921569
Number of CV Scores used in Average:  5


In [543]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [544]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80667892 0.81004902 0.79871324 0.8091299  0.81158088 0.81096814]
Average CV Score:  0.8078533496732025
Number of CV Scores used in Average:  6


In [545]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [546]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80667892 0.81004902 0.81770833 0.81341912 0.80208333 0.81188725
 0.81648284]
Average CV Score:  0.8111869747899159
Number of CV Scores used in Average:  7


In [547]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [548]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80208333 0.80974265 0.80882353 0.80116422 0.79993873 0.80177696
 0.80330882 0.80943627]
Average CV Score:  0.8045343137254901
Number of CV Scores used in Average:  8


In [549]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [550]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81495098 0.80330882 0.79901961 0.80821078 0.81433824 0.81096814
 0.81035539 0.8091299  0.80055147]
Average CV Score:  0.8078703703703705
Number of CV Scores used in Average:  9


In [551]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [552]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81709559 0.80147059 0.80453431 0.81127451 0.80392157 0.79258578
 0.80238971 0.81985294 0.80085784 0.80269608]
Average CV Score:  0.8056678921568627
Number of CV Scores used in Average:  10


##**Bagging**

In [553]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [554]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.98931402964495
Test data accuracy: 0.7555147058823529


In [555]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [556]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81341912 0.79779412]
Average CV Score:  0.8056066176470589
Number of CV Scores used in Average:  2


In [557]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [558]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80759804 0.81893382 0.80882353]
Average CV Score:  0.8117851307189543
Number of CV Scores used in Average:  3


In [559]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [560]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80759804 0.81096814 0.80422794 0.81495098]
Average CV Score:  0.8094362745098039
Number of CV Scores used in Average:  4


In [561]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [562]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81219363 0.80484069 0.8125     0.81801471 0.80300245]
Average CV Score:  0.8101102941176471
Number of CV Scores used in Average:  5


In [563]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [564]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81556373 0.80238971 0.8125     0.80484069 0.81525735 0.80330882]
Average CV Score:  0.8089767156862746
Number of CV Scores used in Average:  6


In [565]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [566]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81709559 0.81678922 0.81341912 0.81096814 0.80790441 0.80698529
 0.81770833]
Average CV Score:  0.8129814425770308
Number of CV Scores used in Average:  7


In [567]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [568]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80606618 0.80729167 0.82107843 0.80637255 0.81464461 0.80330882
 0.79901961 0.8036152 ]
Average CV Score:  0.8076746323529411
Number of CV Scores used in Average:  8


In [569]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [570]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81158088 0.8158701  0.80392157 0.80698529 0.80392157 0.80208333
 0.80300245 0.80208333 0.8192402 ]
Average CV Score:  0.8076320806100218
Number of CV Scores used in Average:  9


In [571]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [572]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.79626225 0.80208333 0.80851716 0.81341912 0.81770833 0.80606618
 0.80943627 0.8002451  0.80606618 0.80667892]
Average CV Score:  0.8066482843137255
Number of CV Scores used in Average:  10


#**ar1 Dataset**

In [573]:
ar1_df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,9,0,1,0,8,7,8,15,20,15,...,0,0,2,0.25,0.00,0,0.00,0.22,0,False
1,21,0,14,1,7,15,12,21,36,27,...,4,1,4,0.57,1.00,1,0.25,0.19,0,False
2,30,0,11,0,19,16,18,50,70,34,...,9,2,9,0.47,1.11,1,0.11,0.30,0,True
3,8,0,2,0,6,4,5,5,10,9,...,1,0,2,0.33,1.00,1,0.50,0.25,0,False
4,7,0,1,0,6,4,5,4,11,9,...,0,0,2,0.33,0.00,2,1.00,0.29,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,32,0,5,0,27,16,11,33,56,27,...,9,1,8,0.30,1.00,0,0.00,0.25,0,False
116,6,0,0,0,6,3,5,5,11,8,...,2,0,3,0.50,1.00,0,0.00,0.50,0,False
117,6,0,0,0,6,6,6,6,11,12,...,2,1,2,0.33,1.00,1,0.50,0.33,0,False
118,27,0,3,0,24,16,9,26,44,25,...,8,3,6,0.25,1.00,1,0.17,0.22,0,False


####**Spiliting Dataset**
training: 80% testing:20%

In [574]:
x = ar1_df.iloc[:,:-1]
y = ar1_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [575]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.875
Confusion Matrix: 
 [[21  0]
 [ 3  0]]


In [576]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [577]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.83333333]
Average CV Score:  0.8333333333333334
Number of CV Scores used in Average:  2


In [578]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [579]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.91666667 0.80555556]
Average CV Score:  0.8796296296296297
Number of CV Scores used in Average:  3


In [580]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [581]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.94444444 0.80555556 0.83333333]
Average CV Score:  0.8541666666666666
Number of CV Scores used in Average:  4


In [582]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [583]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.88888889 0.91666667 0.88888889 0.80555556]
Average CV Score:  0.8666666666666666
Number of CV Scores used in Average:  5


In [584]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [585]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.86111111 0.97222222 0.91666667 0.88888889 0.91666667]
Average CV Score:  0.9120370370370371
Number of CV Scores used in Average:  6


In [586]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [587]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.91666667 0.91666667 0.86111111 0.80555556 0.88888889
 0.88888889]
Average CV Score:  0.884920634920635
Number of CV Scores used in Average:  7


In [588]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [589]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.94444444 0.94444444 0.86111111 0.91666667 0.83333333
 0.97222222 0.88888889]
Average CV Score:  0.9131944444444444
Number of CV Scores used in Average:  8


In [590]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [591]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.86111111 0.83333333 0.86111111 0.94444444 0.97222222
 0.88888889 0.83333333 0.91666667]
Average CV Score:  0.8981481481481483
Number of CV Scores used in Average:  9


In [592]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [593]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80555556 0.94444444 0.83333333 0.80555556 0.86111111 0.94444444
 0.86111111 0.80555556 0.83333333 0.94444444]
Average CV Score:  0.8638888888888887
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [594]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [595]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [596]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[21  0]
 [ 3  0]]


In [597]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.875


In [598]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [599]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.94444444]
Average CV Score:  0.9305555555555556
Number of CV Scores used in Average:  2


In [600]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [601]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.88888889 0.97222222]
Average CV Score:  0.9074074074074074
Number of CV Scores used in Average:  3


In [602]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [603]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.88888889 0.88888889 0.94444444]
Average CV Score:  0.8958333333333333
Number of CV Scores used in Average:  4


In [604]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [605]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.91666667 0.88888889 0.91666667 0.91666667]
Average CV Score:  0.9111111111111111
Number of CV Scores used in Average:  5


In [606]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [607]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.94444444 0.94444444 0.94444444 0.97222222 0.94444444]
Average CV Score:  0.9444444444444445
Number of CV Scores used in Average:  6


In [608]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [609]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.94444444 0.91666667 0.86111111 0.94444444 0.88888889
 0.91666667]
Average CV Score:  0.9087301587301588
Number of CV Scores used in Average:  7


In [610]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [611]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.91666667 0.88888889 0.94444444 0.91666667 0.88888889
 0.91666667 1.        ]
Average CV Score:  0.9201388888888888
Number of CV Scores used in Average:  8


In [612]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [613]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.86111111 0.94444444 0.91666667 0.97222222 0.91666667
 0.94444444 0.88888889 0.91666667]
Average CV Score:  0.9228395061728395
Number of CV Scores used in Average:  9


In [614]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [615]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.88888889 0.88888889 0.94444444 0.91666667 0.88888889
 0.88888889 0.88888889 0.91666667 0.91666667]
Average CV Score:  0.9027777777777777
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [616]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [617]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [618]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[21  0]
 [ 3  0]]


In [619]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.875


In [620]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [621]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80555556 0.80555556]
Average CV Score:  0.8055555555555556
Number of CV Scores used in Average:  2


In [622]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [623]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.88888889 0.80555556]
Average CV Score:  0.8796296296296297
Number of CV Scores used in Average:  3


In [624]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [625]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.80555556 0.91666667 0.86111111]
Average CV Score:  0.8611111111111112
Number of CV Scores used in Average:  4


In [626]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [627]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.91666667 0.94444444 0.91666667 0.80555556]
Average CV Score:  0.888888888888889
Number of CV Scores used in Average:  5


In [628]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [629]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.83333333 0.83333333 0.88888889 0.91666667 0.91666667]
Average CV Score:  0.8750000000000001
Number of CV Scores used in Average:  6


In [630]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [631]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.80555556 0.86111111 0.91666667 0.80555556 0.75       0.88888889
 0.86111111]
Average CV Score:  0.8412698412698413
Number of CV Scores used in Average:  7


In [632]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [633]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.83333333 0.86111111 0.91666667 0.91666667 0.88888889
 0.86111111 0.91666667]
Average CV Score:  0.8854166666666666
Number of CV Scores used in Average:  8


In [634]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [635]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.80555556 0.88888889 0.83333333 0.83333333 0.97222222
 0.75       0.94444444 0.83333333]
Average CV Score:  0.8611111111111112
Number of CV Scores used in Average:  9


In [636]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [637]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.88888889 0.88888889 0.91666667 0.91666667 0.88888889
 0.77777778 0.83333333 0.91666667 0.86111111]
Average CV Score:  0.8722222222222223
Number of CV Scores used in Average:  10


##**Random Forest**

In [638]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [639]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [640]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[21  0]
 [ 3  0]]


In [641]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.875


In [642]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [643]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.94444444]
Average CV Score:  0.9166666666666666
Number of CV Scores used in Average:  2


In [644]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [645]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.86111111 0.97222222]
Average CV Score:  0.9259259259259259
Number of CV Scores used in Average:  3


In [646]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [647]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.97222222 0.94444444 0.88888889]
Average CV Score:  0.9305555555555555
Number of CV Scores used in Average:  4


In [648]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [649]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.94444444 0.88888889 0.80555556 0.94444444]
Average CV Score:  0.8944444444444445
Number of CV Scores used in Average:  5


In [650]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [651]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.94444444 0.88888889 0.88888889 0.86111111 0.88888889]
Average CV Score:  0.8981481481481483
Number of CV Scores used in Average:  6


In [652]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [653]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.91666667 0.91666667 0.94444444 0.94444444 0.91666667
 0.86111111]
Average CV Score:  0.9246031746031748
Number of CV Scores used in Average:  7


In [654]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [655]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.91666667 0.86111111 0.91666667 0.86111111 0.97222222
 0.83333333 0.86111111]
Average CV Score:  0.8993055555555556
Number of CV Scores used in Average:  8


In [656]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [657]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.88888889 0.94444444 0.86111111 0.91666667 0.91666667
 0.94444444 0.94444444 0.91666667]
Average CV Score:  0.9135802469135802
Number of CV Scores used in Average:  9


In [658]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [659]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.97222222 0.91666667 0.80555556 0.94444444 0.86111111
 0.83333333 0.91666667 0.94444444 0.91666667]
Average CV Score:  0.9083333333333332
Number of CV Scores used in Average:  10


##**Bagging**

In [660]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [661]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 0.9895833333333334
Test data accuracy: 0.8333333333333334


In [662]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [663]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.91666667 0.88888889]
Average CV Score:  0.9027777777777777
Number of CV Scores used in Average:  2


In [664]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [665]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.88888889 0.97222222]
Average CV Score:  0.9351851851851851
Number of CV Scores used in Average:  3


In [666]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [667]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.88888889 0.97222222 0.94444444]
Average CV Score:  0.9444444444444444
Number of CV Scores used in Average:  4


In [668]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [669]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.97222222 0.91666667 0.91666667 0.97222222]
Average CV Score:  0.9333333333333332
Number of CV Scores used in Average:  5


In [670]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [671]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.91666667 0.91666667 0.91666667 0.97222222 0.86111111]
Average CV Score:  0.9120370370370369
Number of CV Scores used in Average:  6


In [672]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [673]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86111111 0.91666667 0.91666667 0.86111111 0.91666667 1.
 0.86111111]
Average CV Score:  0.9047619047619049
Number of CV Scores used in Average:  7


In [674]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [675]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.88888889 0.94444444 0.80555556 0.86111111 0.94444444 0.83333333
 0.88888889 0.97222222]
Average CV Score:  0.8923611111111112
Number of CV Scores used in Average:  8


In [676]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [677]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.97222222 0.91666667 0.91666667 0.91666667 0.91666667 0.94444444
 0.88888889 0.91666667 0.91666667]
Average CV Score:  0.9228395061728395
Number of CV Scores used in Average:  9


In [678]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [679]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94444444 0.91666667 0.94444444 0.86111111 0.86111111 0.94444444
 0.86111111 0.88888889 0.86111111 0.94444444]
Average CV Score:  0.9027777777777779
Number of CV Scores used in Average:  10


#**ar3 Dataset**

In [680]:
ar3_df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,3,0,0,0,3,4,6,6,8,10,...,0,0,1,0.33333,0.0000,0,0.000000,0.333330,0,False
1,268,72,22,0,174,125,23,337,484,148,...,94,32,64,0.36782,1.0106,0,0.000000,0.238810,0,False
2,11,2,0,0,9,10,4,15,17,14,...,0,0,1,0.11111,0.0000,2,2.000000,0.090909,0,False
3,9,2,0,0,7,7,4,11,13,11,...,0,0,1,0.14286,0.0000,1,1.000000,0.111110,0,False
4,10,2,0,0,8,7,4,13,15,11,...,0,0,1,0.12500,0.0000,2,2.000000,0.100000,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,173,59,22,6,92,77,31,277,352,108,...,53,14,40,0.43478,1.0189,0,0.000000,0.231210,0,True
58,114,32,12,3,70,46,23,134,180,69,...,27,7,22,0.31429,1.0370,1,0.045455,0.192980,0,False
59,148,41,5,0,102,42,16,147,208,58,...,24,7,18,0.17647,1.0000,0,0.000000,0.121620,0,False
60,10,0,0,0,10,11,7,30,38,18,...,12,6,7,0.70000,1.0000,1,0.142860,0.700000,0,False


####**Spiliting Dataset**
training: 80% testing:20%

In [681]:
x = ar3_df.iloc[:,:-1]
y = ar3_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [682]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.8461538461538461
Confusion Matrix: 
 [[11  1]
 [ 1  0]]


In [683]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [684]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.89473684]
Average CV Score:  0.9210526315789473
Number of CV Scores used in Average:  2


In [685]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [686]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.68421053 0.89473684 0.89473684]
Average CV Score:  0.8245614035087719
Number of CV Scores used in Average:  3


In [687]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [688]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.84210526 0.84210526 0.84210526]
Average CV Score:  0.8421052631578947
Number of CV Scores used in Average:  4


In [689]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [690]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.89473684 1.         0.94736842 0.89473684]
Average CV Score:  0.9368421052631579
Number of CV Scores used in Average:  5


In [691]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [692]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.78947368 0.89473684 0.89473684 0.78947368 0.84210526]
Average CV Score:  0.8596491228070176
Number of CV Scores used in Average:  6


In [693]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [694]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.84210526 0.84210526 0.84210526 0.89473684
 0.89473684]
Average CV Score:  0.8721804511278195
Number of CV Scores used in Average:  7


In [695]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [696]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.89473684 0.78947368 0.89473684 0.84210526
 0.84210526 0.84210526]
Average CV Score:  0.8618421052631579
Number of CV Scores used in Average:  8


In [697]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [698]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73684211 0.89473684 0.89473684 0.94736842 1.         0.94736842
 0.84210526 0.94736842 0.89473684]
Average CV Score:  0.9005847953216374
Number of CV Scores used in Average:  9


In [699]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [700]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.94736842 0.84210526 0.94736842 0.89473684 0.94736842
 0.84210526 0.78947368 0.73684211 0.94736842]
Average CV Score:  0.8736842105263157
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [701]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [702]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [703]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[12  0]
 [ 1  0]]


In [704]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9230769230769231


In [705]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [706]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.78947368]
Average CV Score:  0.868421052631579
Number of CV Scores used in Average:  2


In [707]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [708]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78947368 0.94736842 1.        ]
Average CV Score:  0.912280701754386
Number of CV Scores used in Average:  3


In [709]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [710]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.84210526 0.84210526 0.94736842]
Average CV Score:  0.881578947368421
Number of CV Scores used in Average:  4


In [711]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [712]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.89473684 0.84210526 0.84210526 0.89473684]
Average CV Score:  0.8631578947368421
Number of CV Scores used in Average:  5


In [713]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [714]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.94736842 0.89473684 0.94736842 0.73684211 0.94736842]
Average CV Score:  0.8859649122807017
Number of CV Scores used in Average:  6


In [715]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [716]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.78947368 0.78947368 0.89473684 0.94736842 0.84210526
 0.89473684]
Average CV Score:  0.8721804511278195
Number of CV Scores used in Average:  7


In [717]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [718]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.94736842 0.78947368 0.84210526 0.89473684 0.89473684
 0.94736842 0.84210526]
Average CV Score:  0.881578947368421
Number of CV Scores used in Average:  8


In [719]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [720]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.84210526 0.89473684 0.94736842 0.84210526
 0.94736842 0.94736842 0.84210526]
Average CV Score:  0.8947368421052632
Number of CV Scores used in Average:  9


In [721]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [722]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.89473684 0.94736842 0.94736842 0.89473684 0.84210526
 0.84210526 0.84210526 0.94736842 0.84210526]
Average CV Score:  0.8842105263157893
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [723]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [724]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [725]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[12  0]
 [ 1  0]]


In [726]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9230769230769231


In [727]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [728]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73684211 0.89473684]
Average CV Score:  0.8157894736842105
Number of CV Scores used in Average:  2


In [729]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [730]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.78947368]
Average CV Score:  0.8596491228070174
Number of CV Scores used in Average:  3


In [731]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [732]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.94736842 0.89473684 0.78947368]
Average CV Score:  0.868421052631579
Number of CV Scores used in Average:  4


In [733]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [734]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78947368 0.94736842 0.84210526 0.89473684 0.84210526]
Average CV Score:  0.8631578947368421
Number of CV Scores used in Average:  5


In [735]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [736]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78947368 0.89473684 0.84210526 0.94736842 0.89473684 0.84210526]
Average CV Score:  0.8684210526315789
Number of CV Scores used in Average:  6


In [737]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [738]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78947368 0.84210526 0.89473684 0.84210526 0.94736842 0.89473684
 0.73684211]
Average CV Score:  0.8496240601503758
Number of CV Scores used in Average:  7


In [739]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [740]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.89473684 0.84210526 1.         0.94736842 0.94736842
 0.94736842 0.89473684]
Average CV Score:  0.9144736842105263
Number of CV Scores used in Average:  8


In [741]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [742]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.84210526 0.89473684 0.78947368 0.84210526 0.84210526
 0.89473684 0.68421053 0.89473684]
Average CV Score:  0.8421052631578947
Number of CV Scores used in Average:  9


In [743]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [744]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.89473684 0.94736842 0.89473684 0.78947368 0.89473684
 1.         0.89473684 0.89473684 0.78947368]
Average CV Score:  0.8894736842105264
Number of CV Scores used in Average:  10


##**Random Forest**

In [745]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [746]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [747]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[12  0]
 [ 1  0]]


In [748]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.9230769230769231


In [749]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [750]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.78947368]
Average CV Score:  0.868421052631579
Number of CV Scores used in Average:  2


In [751]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [752]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.78947368 0.89473684]
Average CV Score:  0.8421052631578947
Number of CV Scores used in Average:  3


In [753]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [754]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 1.         0.78947368 0.84210526]
Average CV Score:  0.894736842105263
Number of CV Scores used in Average:  4


In [755]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [756]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.89473684 0.94736842 0.94736842 0.94736842]
Average CV Score:  0.9263157894736842
Number of CV Scores used in Average:  5


In [757]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [758]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.94736842 0.94736842 0.89473684 0.84210526 0.89473684]
Average CV Score:  0.8947368421052629
Number of CV Scores used in Average:  6


In [759]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [760]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.84210526 0.94736842 0.94736842 0.89473684 0.94736842
 0.94736842]
Average CV Score:  0.9172932330827068
Number of CV Scores used in Average:  7


In [761]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [762]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.84210526 0.94736842 0.84210526 0.94736842
 0.89473684 0.94736842]
Average CV Score:  0.9013157894736841
Number of CV Scores used in Average:  8


In [763]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [764]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.89473684 0.89473684 0.78947368 0.94736842 0.84210526
 0.89473684 0.89473684 0.84210526]
Average CV Score:  0.8771929824561403
Number of CV Scores used in Average:  9


In [765]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [766]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.84210526 0.78947368 0.89473684 0.78947368
 0.84210526 0.84210526 1.         0.94736842]
Average CV Score:  0.8736842105263157
Number of CV Scores used in Average:  10


##**Bagging**

In [767]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [768]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 1.0
Test data accuracy: 0.9230769230769231


In [769]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [770]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.84210526]
Average CV Score:  0.868421052631579
Number of CV Scores used in Average:  2


In [771]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [772]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84210526 0.89473684 1.        ]
Average CV Score:  0.912280701754386
Number of CV Scores used in Average:  3


In [773]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [774]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.89473684 0.84210526 0.84210526]
Average CV Score:  0.881578947368421
Number of CV Scores used in Average:  4


In [775]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [776]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.89473684 0.84210526 0.78947368 0.89473684]
Average CV Score:  0.8736842105263157
Number of CV Scores used in Average:  5


In [777]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [778]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.84210526 0.94736842 0.89473684 0.84210526 0.89473684]
Average CV Score:  0.8947368421052629
Number of CV Scores used in Average:  6


In [779]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [780]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         0.89473684 0.78947368 0.89473684 0.94736842 0.89473684
 0.94736842]
Average CV Score:  0.9097744360902256
Number of CV Scores used in Average:  7


In [781]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [782]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.89473684 0.94736842 0.84210526 0.89473684 0.89473684 0.94736842
 0.94736842 0.94736842]
Average CV Score:  0.9144736842105263
Number of CV Scores used in Average:  8


In [783]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [784]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.94736842 0.94736842 1.         0.84210526 0.94736842 1.
 0.84210526 0.89473684 0.84210526]
Average CV Score:  0.9181286549707602
Number of CV Scores used in Average:  9


In [785]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [786]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         0.84210526 0.94736842 1.         0.89473684 0.94736842
 0.94736842 0.94736842 0.89473684 0.84210526]
Average CV Score:  0.9263157894736842
Number of CV Scores used in Average:  10


#**ar4 Dataset**

In [787]:
ar4_df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,53,22,5,0,26,18,16,38,54,34,...,5,2,5,0.19231,1.2,0,0.00,0.094340,1,False
1,25,10,1,0,14,12,14,42,58,26,...,1,0,2,0.14286,1.0,0,0.00,0.080000,2,False
2,73,22,8,1,43,25,12,67,97,37,...,15,7,9,0.20930,1.0,0,0.00,0.123290,0,False
3,69,21,17,0,31,16,9,28,48,25,...,13,6,8,0.25806,1.0,0,0.00,0.115940,0,False
4,42,18,1,0,23,12,10,23,39,22,...,7,1,7,0.30435,1.0,0,0.00,0.166670,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,71,27,9,0,35,19,16,53,80,35,...,5,1,6,0.17143,1.2,0,0.00,0.084507,0,False
102,79,40,5,0,34,23,10,52,79,33,...,9,2,8,0.23529,1.0,2,0.25,0.101270,0,False
103,19,8,0,0,11,8,14,19,29,22,...,1,0,3,0.27273,2.0,0,0.00,0.157890,2,False
104,119,59,13,0,47,20,23,68,107,43,...,10,2,10,0.21277,1.1,2,0.20,0.084034,0,False


####**Spiliting Dataset**
training: 80% testing:20%

In [788]:
x = ar4_df.iloc[:,:-1]
y = ar4_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [789]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.9090909090909091
Confusion Matrix: 
 [[18  1]
 [ 1  2]]


In [790]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [791]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.78125]
Average CV Score:  0.8125
Number of CV Scores used in Average:  2


In [792]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [793]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.84375 0.75   ]
Average CV Score:  0.8125
Number of CV Scores used in Average:  3


In [794]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [795]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.84375 0.84375 0.8125 ]
Average CV Score:  0.828125
Number of CV Scores used in Average:  4


In [796]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [797]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.6875  0.78125 0.65625 0.875  ]
Average CV Score:  0.75625
Number of CV Scores used in Average:  5


In [798]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [799]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.875   0.84375 0.8125  0.875   0.8125  0.84375]
Average CV Score:  0.84375
Number of CV Scores used in Average:  6


In [800]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [801]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.84375 0.78125 0.84375 0.875   0.8125  0.875  ]
Average CV Score:  0.8392857142857143
Number of CV Scores used in Average:  7


In [802]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [803]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.75    0.71875 0.78125 0.71875 0.71875 0.875   0.90625]
Average CV Score:  0.78515625
Number of CV Scores used in Average:  8


In [804]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [805]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75    0.78125 0.8125  0.75    0.84375 0.78125 0.78125 0.8125  0.84375]
Average CV Score:  0.7951388888888888
Number of CV Scores used in Average:  9


In [806]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [807]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.8125  0.875   0.78125 0.8125  0.6875  0.90625 0.875   0.875
 0.8125 ]
Average CV Score:  0.821875
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [808]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [809]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [810]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[18  1]
 [ 2  1]]


In [811]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8636363636363636


In [812]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [813]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.71875 0.90625]
Average CV Score:  0.8125
Number of CV Scores used in Average:  2


In [814]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [815]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.875   0.84375 0.8125 ]
Average CV Score:  0.84375
Number of CV Scores used in Average:  3


In [816]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [817]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125 0.75   0.875  0.875 ]
Average CV Score:  0.828125
Number of CV Scores used in Average:  4


In [818]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [819]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.71875 0.8125  0.78125 0.875   0.8125 ]
Average CV Score:  0.8
Number of CV Scores used in Average:  5


In [820]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [821]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.84375 0.90625 0.84375 0.84375 0.78125]
Average CV Score:  0.8385416666666666
Number of CV Scores used in Average:  6


In [822]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [823]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.9375  0.75    0.875   0.875   0.84375 0.8125 ]
Average CV Score:  0.8482142857142857
Number of CV Scores used in Average:  7


In [824]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [825]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9375  0.90625 1.      0.875   0.875   0.75    0.8125  0.8125 ]
Average CV Score:  0.87109375
Number of CV Scores used in Average:  8


In [826]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [827]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.96875 0.875   0.78125 0.75    0.90625 0.875   0.90625 0.78125 0.875  ]
Average CV Score:  0.8576388888888888
Number of CV Scores used in Average:  9


In [828]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [829]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.71875 0.78125 0.875   0.90625 0.84375 0.8125  0.8125  0.96875 0.8125
 0.90625]
Average CV Score:  0.84375
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [830]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [831]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [832]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[17  2]
 [ 2  1]]


In [833]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8181818181818182


In [834]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [835]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.84375]
Average CV Score:  0.8125
Number of CV Scores used in Average:  2


In [836]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [837]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125 0.75   0.75  ]
Average CV Score:  0.7708333333333334
Number of CV Scores used in Average:  3


In [838]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [839]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.78125 0.75    0.90625]
Average CV Score:  0.8046875
Number of CV Scores used in Average:  4


In [840]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [841]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.71875 0.84375 0.8125  0.75    0.75   ]
Average CV Score:  0.775
Number of CV Scores used in Average:  5


In [842]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [843]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.71875 0.84375 0.8125  0.875   0.8125 ]
Average CV Score:  0.8125
Number of CV Scores used in Average:  6


In [844]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [845]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.875   0.71875 0.6875  0.71875 0.65625 0.8125 ]
Average CV Score:  0.7544642857142857
Number of CV Scores used in Average:  7


In [846]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [847]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.84375 0.71875 0.71875 0.75    0.78125 0.875   0.78125]
Average CV Score:  0.78515625
Number of CV Scores used in Average:  8


In [848]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [849]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.71875 0.78125 0.875   0.875   0.75    0.6875  0.78125 0.84375]
Average CV Score:  0.7951388888888888
Number of CV Scores used in Average:  9


In [850]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [851]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90625 0.71875 0.90625 0.65625 0.75    0.75    0.78125 0.875   0.84375
 0.75   ]
Average CV Score:  0.79375
Number of CV Scores used in Average:  10


##**Random Forest**

In [852]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [853]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [854]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[18  1]
 [ 2  1]]


In [855]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8636363636363636


In [856]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [857]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.84375]
Average CV Score:  0.84375
Number of CV Scores used in Average:  2


In [858]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [859]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.875   0.84375 0.8125 ]
Average CV Score:  0.84375
Number of CV Scores used in Average:  3


In [860]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [861]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.90625 0.84375 0.84375]
Average CV Score:  0.8515625
Number of CV Scores used in Average:  4


In [862]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [863]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.84375 0.84375 0.8125  0.6875 ]
Average CV Score:  0.79375
Number of CV Scores used in Average:  5


In [864]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [865]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.78125 0.78125 0.8125  0.84375 0.75   ]
Average CV Score:  0.7916666666666666
Number of CV Scores used in Average:  6


In [866]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [867]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.78125 0.84375 0.875   0.78125 0.78125 0.84375]
Average CV Score:  0.8214285714285714
Number of CV Scores used in Average:  7


In [868]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [869]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.875   0.875   0.875   0.8125  0.84375 0.75    0.84375]
Average CV Score:  0.8359375
Number of CV Scores used in Average:  8


In [870]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [871]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.78125 0.90625 0.84375 0.84375 0.8125  0.90625 0.8125  0.8125 ]
Average CV Score:  0.8402777777777778
Number of CV Scores used in Average:  9


In [872]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [873]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.875   0.78125 0.71875 0.96875 0.8125  0.90625 0.8125  0.75
 0.84375]
Average CV Score:  0.828125
Number of CV Scores used in Average:  10


##**Bagging**

In [874]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [875]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 1.0
Test data accuracy: 0.7727272727272727


In [876]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [877]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.8125 ]
Average CV Score:  0.796875
Number of CV Scores used in Average:  2


In [878]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [879]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9375  0.84375 0.90625]
Average CV Score:  0.8958333333333334
Number of CV Scores used in Average:  3


In [880]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [881]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.75    0.84375 0.90625 0.71875]
Average CV Score:  0.8046875
Number of CV Scores used in Average:  4


In [882]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [883]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90625 0.90625 0.75    0.9375  0.875  ]
Average CV Score:  0.875
Number of CV Scores used in Average:  5


In [884]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [885]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.84375 0.84375 0.90625 0.84375 0.8125  0.8125 ]
Average CV Score:  0.84375
Number of CV Scores used in Average:  6


In [886]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [887]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.78125 0.875   0.90625 0.84375 0.71875 0.84375 0.78125]
Average CV Score:  0.8214285714285714
Number of CV Scores used in Average:  7


In [888]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [889]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90625 0.8125  0.9375  0.71875 0.8125  0.8125  0.84375 0.84375]
Average CV Score:  0.8359375
Number of CV Scores used in Average:  8


In [890]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [891]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9375  0.875   0.71875 0.84375 0.84375 0.8125  0.875   0.84375 0.78125]
Average CV Score:  0.8368055555555556
Number of CV Scores used in Average:  9


In [892]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [893]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8125  0.84375 0.75    0.8125  0.9375  0.90625 0.9375  0.8125  0.9375
 0.75   ]
Average CV Score:  0.85
Number of CV Scores used in Average:  10


#**ar5 Dataset**

In [894]:
ar5_df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,16,6,2,1,8,13,6,18,20,19,...,0,0,2,0.250000,0.0000,1,0.500000,0.125000,1,False
1,31,12,3,2,16,18,9,31,42,27,...,5,0,6,0.375000,1.0000,1,0.166670,0.193550,0,False
2,477,104,89,2,284,150,29,482,699,179,...,116,25,93,0.327460,1.0172,4,0.043011,0.194970,0,True
3,11,2,0,0,9,10,4,15,17,14,...,0,0,1,0.111110,0.0000,2,2.000000,0.090909,0,False
4,9,2,0,0,7,7,4,11,13,11,...,0,0,1,0.142860,0.0000,1,1.000000,0.111110,0,False
5,10,2,0,0,8,7,4,13,15,11,...,0,0,1,0.125000,0.0000,2,2.000000,0.100000,0,False
6,5,0,0,0,5,5,3,5,9,8,...,0,0,1,0.200000,0.0000,3,3.000000,0.200000,0,False
7,28,5,1,0,22,18,12,49,53,30,...,3,1,4,0.181820,1.3333,2,0.500000,0.142860,1,False
8,26,6,0,0,20,16,11,41,44,27,...,1,0,3,0.150000,2.0000,2,0.666670,0.115380,0,False
9,15,4,0,0,11,11,5,19,22,16,...,0,0,1,0.090909,0.0000,7,7.000000,0.066667,1,False


####**Spiliting Dataset**
training: 80% testing:20%

In [895]:
x = ar5_df.iloc[:,:-1]
y = ar5_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [896]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  1.0
Confusion Matrix: 
 [[5 0]
 [0 2]]


In [897]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [898]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.90909091]
Average CV Score:  0.9090909090909091
Number of CV Scores used in Average:  2


In [899]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [900]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 1.        ]
Average CV Score:  0.9090909090909092
Number of CV Scores used in Average:  3


In [901]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [902]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         0.81818182 0.81818182 0.90909091]
Average CV Score:  0.8863636363636365
Number of CV Scores used in Average:  4


In [903]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [904]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 1.         0.54545455 0.81818182 0.81818182]
Average CV Score:  0.7818181818181819
Number of CV Scores used in Average:  5


In [905]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [906]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         1.         0.81818182 0.81818182 0.72727273 0.81818182]
Average CV Score:  0.8636363636363638
Number of CV Scores used in Average:  6


In [907]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [908]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         0.90909091 0.81818182 0.90909091 0.81818182 1.
 0.81818182]
Average CV Score:  0.8961038961038962
Number of CV Scores used in Average:  7


In [909]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [910]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.72727273 0.81818182 0.81818182 0.90909091 0.90909091
 0.72727273 0.90909091]
Average CV Score:  0.8295454545454545
Number of CV Scores used in Average:  8


In [911]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [912]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 0.90909091 0.90909091 0.90909091 0.90909091
 0.72727273 0.90909091 0.81818182]
Average CV Score:  0.8686868686868687
Number of CV Scores used in Average:  9


In [913]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [914]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         0.81818182 0.81818182 1.         0.72727273 1.
 0.90909091 0.81818182 0.90909091 1.        ]
Average CV Score:  0.9
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [915]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [916]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [917]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[5 0]
 [2 0]]


In [918]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.7142857142857143


In [919]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [920]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.54545455]
Average CV Score:  0.6363636363636364
Number of CV Scores used in Average:  2


In [921]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [922]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.63636364 0.90909091 0.72727273]
Average CV Score:  0.7575757575757575
Number of CV Scores used in Average:  3


In [923]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [924]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.63636364 0.90909091 0.81818182]
Average CV Score:  0.7954545454545455
Number of CV Scores used in Average:  4


In [925]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [926]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.72727273 0.63636364 0.72727273 0.72727273]
Average CV Score:  0.7090909090909092
Number of CV Scores used in Average:  5


In [927]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [928]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.81818182 0.72727273 0.81818182 0.72727273 0.54545455]
Average CV Score:  0.7272727272727272
Number of CV Scores used in Average:  6


In [929]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [930]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.81818182 0.72727273 0.63636364 0.90909091 0.90909091
 0.90909091]
Average CV Score:  0.8181818181818182
Number of CV Scores used in Average:  7


In [931]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [932]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 0.81818182 0.63636364 0.72727273 0.81818182
 0.63636364 0.54545455]
Average CV Score:  0.7386363636363635
Number of CV Scores used in Average:  8


In [933]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [934]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.63636364 0.81818182 0.81818182 0.72727273 0.72727273 0.72727273
 0.72727273 0.72727273 0.72727273]
Average CV Score:  0.7373737373737375
Number of CV Scores used in Average:  9


In [935]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [936]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.72727273 0.81818182 0.72727273 0.90909091 0.90909091
 0.63636364 0.63636364 0.81818182 0.72727273]
Average CV Score:  0.7727272727272727
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [937]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [938]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [939]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[5 0]
 [2 0]]


In [940]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.7142857142857143


In [941]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [942]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.90909091]
Average CV Score:  0.8181818181818181
Number of CV Scores used in Average:  2


In [943]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [944]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 0.90909091]
Average CV Score:  0.8787878787878788
Number of CV Scores used in Average:  3


In [945]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [946]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.90909091 0.72727273 0.63636364]
Average CV Score:  0.7499999999999999
Number of CV Scores used in Average:  4


In [947]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [948]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.90909091 0.63636364 0.72727273 0.90909091]
Average CV Score:  0.8181818181818181
Number of CV Scores used in Average:  5


In [949]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [950]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.81818182 0.81818182 1.         0.81818182 0.90909091]
Average CV Score:  0.8787878787878788
Number of CV Scores used in Average:  6


In [951]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [952]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.90909091 0.81818182 0.72727273 0.81818182 0.90909091
 0.63636364]
Average CV Score:  0.8181818181818182
Number of CV Scores used in Average:  7


In [953]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [954]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.81818182 0.72727273 0.90909091 0.81818182 0.90909091
 0.81818182 0.90909091]
Average CV Score:  0.8522727272727273
Number of CV Scores used in Average:  8


In [955]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [956]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.81818182 0.72727273 0.72727273 0.81818182 0.90909091
 0.72727273 0.90909091 0.81818182]
Average CV Score:  0.8181818181818181
Number of CV Scores used in Average:  9


In [957]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [958]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.63636364 0.72727273 0.90909091 0.81818182 0.72727273
 0.90909091 0.81818182 0.72727273 0.72727273]
Average CV Score:  0.7727272727272727
Number of CV Scores used in Average:  10


##**Random Forest**

In [959]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [960]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [961]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[5 0]
 [1 1]]


In [962]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8571428571428571


In [963]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [964]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.72727273]
Average CV Score:  0.7727272727272727
Number of CV Scores used in Average:  2


In [965]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [966]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 0.81818182]
Average CV Score:  0.8484848484848485
Number of CV Scores used in Average:  3


In [967]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [968]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.63636364 0.81818182 0.81818182]
Average CV Score:  0.7954545454545455
Number of CV Scores used in Average:  4


In [969]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [970]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.63636364 0.81818182 0.90909091 0.81818182 0.54545455]
Average CV Score:  0.7454545454545455
Number of CV Scores used in Average:  5


In [971]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [972]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.81818182 0.63636364 0.72727273 0.90909091 0.81818182]
Average CV Score:  0.787878787878788
Number of CV Scores used in Average:  6


In [973]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [974]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.90909091 1.         0.72727273 0.81818182 0.90909091
 0.90909091]
Average CV Score:  0.8831168831168832
Number of CV Scores used in Average:  7


In [975]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [976]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 0.81818182 0.81818182 0.63636364 0.81818182
 0.81818182 0.81818182]
Average CV Score:  0.8068181818181819
Number of CV Scores used in Average:  8


In [977]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [978]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         1.         0.90909091 0.72727273 0.72727273 0.54545455
 0.90909091 1.         0.81818182]
Average CV Score:  0.8484848484848485
Number of CV Scores used in Average:  9


In [979]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [980]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [1.         1.         0.63636364 0.72727273 0.81818182 0.72727273
 0.81818182 0.90909091 0.81818182 0.81818182]
Average CV Score:  0.8272727272727274
Number of CV Scores used in Average:  10


##**Bagging**

In [981]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [982]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 1.0
Test data accuracy: 0.8571428571428571


In [983]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [984]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.90909091]
Average CV Score:  0.9090909090909091
Number of CV Scores used in Average:  2


In [985]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [986]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.63636364 0.81818182 0.72727273]
Average CV Score:  0.7272727272727272
Number of CV Scores used in Average:  3


In [987]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [988]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.81818182 0.72727273 0.81818182]
Average CV Score:  0.7954545454545455
Number of CV Scores used in Average:  4


In [989]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [990]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.63636364 0.90909091 0.81818182 0.81818182]
Average CV Score:  0.781818181818182
Number of CV Scores used in Average:  5


In [991]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [992]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 0.90909091 1.         0.72727273 0.81818182 0.90909091]
Average CV Score:  0.8636363636363638
Number of CV Scores used in Average:  6


In [993]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [994]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.81818182 1.         0.63636364 0.90909091 0.90909091 0.90909091
 0.81818182]
Average CV Score:  0.8571428571428571
Number of CV Scores used in Average:  7


In [995]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [996]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 0.72727273 0.72727273 0.90909091 0.90909091 0.72727273
 0.81818182 0.90909091]
Average CV Score:  0.8295454545454545
Number of CV Scores used in Average:  8


In [997]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [998]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.72727273 0.90909091 0.54545455 0.81818182 0.81818182 0.81818182
 0.81818182 0.90909091 0.81818182]
Average CV Score:  0.797979797979798
Number of CV Scores used in Average:  9


In [999]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1000]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.90909091 1.         0.72727273 0.63636364 0.54545455 0.72727273
 0.54545455 0.81818182 0.81818182 0.63636364]
Average CV Score:  0.7363636363636366
Number of CV Scores used in Average:  10


#**ar6 Dataset**

In [1001]:
ar6_df

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,13,0,1,0,12,10,8,21,26,18,...,0,0,2,0.17,0.00,0.0,0.00,0.15,0.0,False
1,20,0,12,0,8,14,12,21,34,26,...,3,0,4,0.50,1.00,0.0,0.00,0.20,0.0,False
2,40,0,17,0,23,20,18,64,90,38,...,11,2,11,0.48,1.09,1.0,0.09,0.28,0.0,False
3,8,0,3,0,5,7,13,13,21,20,...,1,0,3,0.60,2.00,0.0,0.00,0.38,1.0,False
4,10,0,2,0,8,4,5,5,10,9,...,1,0,2,0.25,1.00,1.0,0.50,0.20,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,27,0,10,0,17,15,9,36,48,24,...,4,1,4,0.24,1.00,5.0,1.25,0.15,0.0,False
96,9,0,0,0,9,7,3,8,15,10,...,0,0,1,0.11,0.00,6.0,6.00,0.11,0.0,False
97,10,0,7,0,3,7,3,7,15,10,...,0,0,1,0.33,0.00,6.0,6.00,0.10,0.0,False
98,51,2,17,0,32,26,9,44,69,35,...,6,0,7,0.22,1.00,8.0,1.14,0.14,0.0,False


####**Spiliting Dataset**
training: 80% testing:20%

In [1002]:
x = ar6_df.iloc[:,:-1]
y = ar6_df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=0)

from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

###**Logistic Regression**

In [1003]:
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0)  
classifier.fit(x_train, y_train) 

y_pred= classifier.predict(x_test)

score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,y_pred)
print("Confusion Matrix: \n",cm)

Test Accuracy Score:  0.85
Confusion Matrix: 
 [[16  1]
 [ 2  1]]


In [1004]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1005]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.73333333]
Average CV Score:  0.8
Number of CV Scores used in Average:  2


In [1006]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1007]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.96666667 0.86666667 0.83333333]
Average CV Score:  0.888888888888889
Number of CV Scores used in Average:  3


In [1008]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1009]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.86666667 0.93333333 0.73333333]
Average CV Score:  0.8583333333333334
Number of CV Scores used in Average:  4


In [1010]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1011]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8        0.8        0.86666667 0.9        0.83333333]
Average CV Score:  0.8400000000000001
Number of CV Scores used in Average:  5


In [1012]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1013]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.83333333 0.86666667 0.93333333 0.8        0.8       ]
Average CV Score:  0.85
Number of CV Scores used in Average:  6


In [1014]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1015]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.76666667 0.83333333 0.9        0.9        0.8
 0.9       ]
Average CV Score:  0.8571428571428571
Number of CV Scores used in Average:  7


In [1016]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1017]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.86666667 0.8        0.86666667 0.86666667 0.86666667
 0.9        0.83333333]
Average CV Score:  0.8583333333333334
Number of CV Scores used in Average:  8


In [1018]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1019]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 1.         0.9        0.86666667 0.73333333 0.9
 0.8        0.86666667 0.9       ]
Average CV Score:  0.8703703703703705
Number of CV Scores used in Average:  9


In [1020]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [1021]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93333333 0.8        0.83333333 0.63333333 0.9        0.8
 0.86666667 0.86666667 0.86666667 0.93333333]
Average CV Score:  0.8433333333333334
Number of CV Scores used in Average:  10


###**Support Vector Machine Algorithm**

In [1022]:
from sklearn.svm import SVC # "Support vector classifier"  
classifier = SVC(kernel='rbf', random_state=0)  
classifier.fit(x_train, y_train)  

SVC(random_state=0)

In [1023]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [1024]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[17  0]
 [ 3  0]]


In [1025]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.85


In [1026]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1027]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.73333333]
Average CV Score:  0.8
Number of CV Scores used in Average:  2


In [1028]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1029]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.96666667 0.86666667]
Average CV Score:  0.888888888888889
Number of CV Scores used in Average:  3


In [1030]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1031]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8        0.86666667 0.93333333 0.93333333]
Average CV Score:  0.8833333333333333
Number of CV Scores used in Average:  4


In [1032]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1033]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.9        0.83333333 0.9        0.9       ]
Average CV Score:  0.8866666666666667
Number of CV Scores used in Average:  5


In [1034]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1035]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.86666667 0.86666667 0.9        0.93333333 0.8       ]
Average CV Score:  0.8666666666666667
Number of CV Scores used in Average:  6


In [1036]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1037]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.9        0.9        0.8        0.93333333 0.73333333
 0.83333333]
Average CV Score:  0.8476190476190476
Number of CV Scores used in Average:  7


In [1038]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1039]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76666667 0.9        0.93333333 0.86666667 0.8        0.76666667
 0.83333333 0.83333333]
Average CV Score:  0.8375
Number of CV Scores used in Average:  8


In [1040]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1041]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.93333333 0.9        0.9        0.83333333 0.86666667
 0.86666667 0.9        0.9       ]
Average CV Score:  0.8814814814814815
Number of CV Scores used in Average:  9


In [1042]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1043]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.86666667 0.83333333 0.86666667 0.86666667 0.83333333
 0.9        0.86666667 0.83333333 0.9       ]
Average CV Score:  0.8666666666666666
Number of CV Scores used in Average:  10


##**Decision Tree Algorithm**

In [1044]:
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(x_train, y_train)  

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [1045]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [1046]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 
print(cm)

[[14  3]
 [ 2  1]]


In [1047]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.75


In [1048]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1049]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.66666667]
Average CV Score:  0.75
Number of CV Scores used in Average:  2


In [1050]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1051]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73333333 0.7        0.8       ]
Average CV Score:  0.7444444444444445
Number of CV Scores used in Average:  3


In [1052]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1053]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.73333333 0.76666667 0.76666667]
Average CV Score:  0.775
Number of CV Scores used in Average:  4


In [1054]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1055]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.63333333 0.76666667 0.83333333 0.8        0.86666667]
Average CV Score:  0.78
Number of CV Scores used in Average:  5


In [1056]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1057]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.63333333 0.63333333 0.7        0.66666667 0.76666667]
Average CV Score:  0.7055555555555556
Number of CV Scores used in Average:  6


In [1058]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1059]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.6        0.76666667 0.66666667 0.8        0.9        0.7
 0.76666667]
Average CV Score:  0.7428571428571428
Number of CV Scores used in Average:  7


In [1060]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1061]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76666667 0.63333333 0.8        0.86666667 0.76666667 0.86666667
 0.76666667 0.8       ]
Average CV Score:  0.7833333333333333
Number of CV Scores used in Average:  8


In [1062]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1063]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76666667 0.73333333 0.73333333 0.76666667 0.83333333 0.8
 0.83333333 0.76666667 0.76666667]
Average CV Score:  0.7777777777777778
Number of CV Scores used in Average:  9


In [1064]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1065]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.8        0.8        0.63333333 0.86666667 0.73333333
 0.73333333 0.9        0.83333333 0.73333333]
Average CV Score:  0.79
Number of CV Scores used in Average:  10


##**Random Forest**

In [1066]:
#Fitting Decision Tree classifier to the training set  
from sklearn.ensemble import RandomForestClassifier  
classifier= RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [1067]:
#Predicting the test set result  
y_pred= classifier.predict(x_test)  

In [1068]:
#Creating the Confusion matrix  
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred)  
print(cm)

[[16  1]
 [ 3  0]]


In [1069]:
score_ = accuracy_score(y_test, y_pred)
print("Test Accuracy Score: ", score_)

Test Accuracy Score:  0.8


In [1070]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1071]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8        0.76666667]
Average CV Score:  0.7833333333333334
Number of CV Scores used in Average:  2


In [1072]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1073]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93333333 0.83333333 0.93333333]
Average CV Score:  0.9
Number of CV Scores used in Average:  3


In [1074]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1075]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.9        0.83333333 0.9       ]
Average CV Score:  0.8666666666666667
Number of CV Scores used in Average:  4


In [1076]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1077]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.83333333 0.9        0.83333333 0.86666667]
Average CV Score:  0.8600000000000001
Number of CV Scores used in Average:  5


In [1078]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1079]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76666667 0.9        0.9        0.8        0.86666667 0.86666667]
Average CV Score:  0.8500000000000002
Number of CV Scores used in Average:  6


In [1080]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1081]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.83333333 0.93333333 0.9        0.86666667 0.83333333
 0.7       ]
Average CV Score:  0.8476190476190476
Number of CV Scores used in Average:  7


In [1082]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1083]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.9        0.86666667 0.8        0.9        0.83333333
 0.86666667 0.8       ]
Average CV Score:  0.8500000000000001
Number of CV Scores used in Average:  8


In [1084]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1085]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.8        0.86666667 0.83333333 0.8        0.86666667
 0.7        0.83333333 0.83333333]
Average CV Score:  0.825925925925926
Number of CV Scores used in Average:  9


In [1086]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1087]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.83333333 0.86666667 0.76666667 0.93333333 0.83333333 0.73333333
 0.86666667 0.8        0.9        0.93333333]
Average CV Score:  0.8466666666666667
Number of CV Scores used in Average:  10


##**Bagging**

In [1088]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=21)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=21)

In [1089]:
y_pred = dtree.predict(x_test)

print("Train data accuracy:",accuracy_score(y_true = y_train, y_pred = dtree.predict(x_train)))
print("Test data accuracy:",accuracy_score(y_true = y_test, y_pred = y_pred))

Train data accuracy: 1.0
Test data accuracy: 0.7


In [1090]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 2)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1091]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.83333333]
Average CV Score:  0.8500000000000001
Number of CV Scores used in Average:  2


In [1092]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 3)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1093]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.76666667 0.8        0.86666667]
Average CV Score:  0.8111111111111112
Number of CV Scores used in Average:  3


In [1094]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 4)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1095]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.93333333 0.76666667 0.73333333]
Average CV Score:  0.8333333333333334
Number of CV Scores used in Average:  4


In [1096]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 5)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1097]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86666667 0.83333333 0.83333333 0.83333333 0.8       ]
Average CV Score:  0.8333333333333334
Number of CV Scores used in Average:  5


In [1098]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 6)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1099]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.93333333 0.8        0.83333333 0.86666667 0.9        0.83333333]
Average CV Score:  0.8611111111111112
Number of CV Scores used in Average:  6


In [1100]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 7)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1101]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.8        0.86666667 0.8        1.         0.8        0.83333333
 0.86666667]
Average CV Score:  0.8523809523809524
Number of CV Scores used in Average:  7


In [1102]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 8)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1103]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9        0.83333333 0.8        0.86666667 0.83333333 0.86666667
 0.8        0.8       ]
Average CV Score:  0.8375000000000001
Number of CV Scores used in Average:  8


In [1104]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 9)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1105]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73333333 0.73333333 0.93333333 0.83333333 0.8        0.76666667
 0.8        0.8        0.93333333]
Average CV Score:  0.8148148148148149
Number of CV Scores used in Average:  9


In [1106]:
ss = ShuffleSplit(train_size=0.7, test_size=0.3, n_splits = 10)

scores = cross_val_score(classifier, x, y, cv = ss)

In [1107]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.73333333 0.8        0.93333333 0.86666667 0.9        0.83333333
 0.8        0.86666667 0.8        0.83333333]
Average CV Score:  0.8366666666666667
Number of CV Scores used in Average:  10
